In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math
model = ConcreteModel()

#目次_下記の順で記述している
#経営条件（前提，変数設定，制約）
#牛群構成部門（前提， 変数設定， 制約）
#飼料生産部門（前提， 変数設定， 制約）
#飼料設計部門（前提， 変数設定， 制約）
#機械調達部門（前提， 変数設定， 制約）
#飼養管理部門（前提， 変数設定， 制約）
#目標関数（所得最大化）
#シミュレーション条件（飼料用稲の収穫面積設定）



#経営条件_前提:
milk=28.8 #乳量
fat=3.89 #乳脂率
feed_type=0 #分離で1, TMRで0
birth_count=2 #産次数1,2
calving_interval=13 
delivery_day=(calving_interval*30-60)/2 #分娩後日数
w=650 #体重
number_of_family_core_labor=2 #基幹的家族労働
number_of_family_sub_labor=0 #補助的家族労働
cost_of_hierd_labor=6500000 #通年雇用者の給与額
farming_area=1000 #自作地面積
yield_corn=400 #トウモロコシaあたり収量kg
yield_italian=110 #イタリアンaあたり収量kg
yield_fodder_rice=110 #飼料用稲aあたり収量kg
operatable_term_corn_harvest=10 #旬あたり作業日数
sunny_rate=0.7 #晴れの確率

#経営条件_変数設定：
model.revenue=Var(initialize=1,within=NonNegativeReals) #乳代+雑収入
model.miscRev=Var(initialize=1,within=NonNegativeReals) #雑収入
model.cost_feed=Var(initialize=1,within=NonNegativeReals) #購入飼料費
model.cost=Var(initialize=1,within=NonNegativeReals) #農業経営費
model.cost_fixed=Var(initialize=1,within=NonNegativeReals) #機械焼却費
model.cost_raised_feed=Var(initialize=1,within=NonNegativeReals) #育成牛飼料費
model.cost_raised_fixed=Var(initialize=1,within=NonNegativeReals) #育成牛牛舎焼却費
model.cost_cattle_buy=Var(initialize=1,within=NonNegativeReals) #導入牛経費
model.number_of_hierd_labor_year=Var(initialize=0,within=NonNegativeIntegers) #常雇人数

#経営条件_制約:年間雇用労働
model.const_hired_labor_upper=Constraint(expr=model.number_of_hierd_labor_year==0)









#牛群構成部門_前提:
# calving_interval=13 #分娩間隔→経営条件に移動
cattle_availability=(calving_interval*30-60)/(calving_interval*30)
cattle_update=0.3 #乳牛更新率
price_milk=99.91 #生乳価格
price_calf_xx=150000 #雌子牛価格
price_calf_xy=200000 #雄子牛価格
price_cattle=900000 #導入牛価格
variable_cost_of_cattle=171967 #租税公課, 素畜費，農薬衛生費, 諸材料費, 修繕費，動力光熱費，農業共済掛け金, 雑費
variable_cost_of_raised=26645.76318  #育成牛舎
variable_cost_of_keisan=45785.96876 #機械倉庫，堆肥舎，牛舎

#牛群構成部門_変数設定：頭数
model.cattle=Var(initialize=82,within=NonNegativeReals) #経産牛
model.cattle_milking=Var(initialize=70,within=NonNegativeReals) #搾乳牛
model.cattle_dry=Var(initialize=10,within=NonNegativeReals) #乾乳牛
model.xx_rate=Var(initialize=0.5,within=NonNegativeReals) #雌比率
model.calf_xy=Var(initialize=60,within=NonNegativeReals) #子牛雄
model.calf_xx=Var(initialize=15,within=NonNegativeReals) #子牛雌
model.cattle_raised=Var(initialize=15,within=NonNegativeReals) #育成牛向け
model.cattle_raised_total=Var(initialize=30,within=NonNegativeReals) #育成牛合計
model.cattle_buy=Var(initialize=5,within=NonNegativeReals) #導入牛

#牛群構成部門_制約: 平川輝夫（1997）「畜産経営の生き残り戦略」p.117を参考にした．
model.const_available=Constraint(expr= model.cattle_milking==model.cattle*cattle_availability)
model.const_dry=Constraint(expr=model.cattle_dry==model.cattle-model.cattle_milking)
model.const_xy=Constraint(expr=model.calf_xy==model.cattle*12/calving_interval*(1-model.xx_rate))
model.const_xx=Constraint(expr=model.calf_xx==model.cattle*12/calving_interval*model.xx_rate)
model.const_raised=Constraint(expr=model.cattle_raised<=model.calf_xx) 
model.const_raised_total=Constraint(expr=model.cattle_raised_total==model.cattle_raised*2) #育成期間2年
model.const_update=Constraint(expr=model.cattle*cattle_update==model.cattle_raised+model.cattle_buy)
model.const_buy_milking_cattle=Constraint(expr=model.cattle_buy==model.cattle*cattle_update*0.3) #導入牛は更新する牛の30%
model.const_cattle=Constraint(expr=model.cattle>=1) #経産牛は１頭以上










#飼料生産部門_前提：可変費用（資材費・雇用労賃・委託費）
variable_cost_of_italian=873.7635561 #aあたりイタリアン資材費（生産費調査種子費+生産費調査肥料費+車両関係費+ラップ）
variable_cost_of_rice_silage=644.9635561 #aあたり飼料稲WCS資材費（燃料+ラップ）
variable_cost_of_corn=962.8459091 #aあたりトウモロコシ資材費（生産費調査種子費+生産費調査肥料費+車両関係費）
cost_of_day_labor=1500 #トウモロコシ収穫の日雇い雇用の労賃（円/h）
cost_of_contract_corn_harvest=2257 #トウモロコシ収穫の委託費円/a
# cost_of_contract_corn_harvest=2370 #トウモロコシ収穫の委託費円/a, 自家収穫への移行点
cost_of_contract_corn_herbicide=155 #市の除草作業料金

#飼料生産部門_前提：技術係数（1時間で作業可能な面積）
operation_ef_grass_rotary_seed=500/5.5 #播種（イタリアン）
operation_ef_grass_press=800/5.5 #鎮圧（イタリアン）
operation_ef_grass_reap=1000/5.5 #刈取（イタリアン・飼料稲）
operation_ef_grass_reverse=1000/5.5 #反転（イタリアン・飼料稲）
operation_ef_grass_gather=1000/5.5 #集草（イタリアン・飼料稲）
operation_ef_grass_role=500/5.5 #ロール（イタリアン・飼料稲）
operation_ef_grass_rapping=(100+150)/2/2*10/5.5 #ラッピング（イタリアン・飼料稲）
operation_ef_corn_compost_spread=800/4/5.5 #堆肥散布（トウモロコシ）
operation_ef_corn_plow=800/3/5.5 #プラウ（トウモロコシ）
operation_ef_corn_rotary=500/5.5 #ロータリー（トウモロコシ）
operation_ef_corn_plant=800/2/5.5 #播種（トウモロコシ）
operation_ef_corn_press=800/5.5 #鎮圧（トウモロコシ）
operation_ef_corn_herbicide1=800/1.5/7 #除草作業1回目（トウモロコシ）
operation_ef_corn_herbicide2=800/1.5/7 #除草作業2回目（トウモロコシ）
operation_ef_corn_harvest3=1000/7/8 #自家収穫（トウモロコシ， 3条ケンパー, ３人一組作業）
operation_ef_corn_harvest=900/5/5.5 #コントラ収穫（トウモロコシ， 自走式， 4人一組作業）


#飼料生産部門_変数設定：各作業の組作業ユニット数
model.number_of_unit_italian_reap=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの刈取（4月中+下旬）
model.number_of_unit_italian_reverse=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの反転（4月中+下旬）
model.number_of_unit_italian_gather=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの集草（4月中+下旬）
model.number_of_unit_italian_roll=Var(initialize=1,within=NonNegativeIntegers) #イタリアンのロール成形（4月中+下旬）
model.number_of_unit_italian_rapping=Var(initialize=1,within=NonNegativeIntegers) #イタリアンのラッピング（4月中+下旬）
model.number_of_unit_italian_plant1=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの播種（10月上旬）
model.number_of_unit_italian_plant2=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの播種（10月中旬）
model.number_of_unit_italian_roller1=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの鎮圧（10月上旬）
model.number_of_unit_italian_roller2=Var(initialize=1,within=NonNegativeIntegers) #イタリアンの鎮圧（10月中旬）
model.number_of_unit_rice_silage_reap=Var(initialize=1,within=NonNegativeIntegers) #飼料用稲の刈取（7月下旬+8月上旬+9月上旬）
model.number_of_unit_rice_silage_reverse=Var(initialize=1,within=NonNegativeIntegers) #飼料用稲の反転（7月下旬+8月上旬+9月上旬）
model.number_of_unit_rice_silage_gather=Var(initialize=1,within=NonNegativeIntegers) #飼料用稲の集草（7月下旬+8月上旬+9月上旬）
model.number_of_unit_rice_silage_roll=Var(initialize=1,within=NonNegativeIntegers) #飼料用稲のロール成形（7月下旬+8月上旬+9月上旬）
model.number_of_unit_rice_silage_rapping=Var(initialize=1,within=NonNegativeIntegers) #飼料用稲のラッピング（7月下旬+8月上旬+9月上旬）
model.number_of_unit_corn_compost_spread=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシの堆肥散布（5月中+下旬）
model.number_of_unit_corn_plow=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシのプラウ（5月中+下旬）
model.number_of_unit_corn_rotary=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシのロータリー（5月中+下旬）
model.number_of_unit_corn_plant=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシの播種（5月中+下旬）
model.number_of_unit_corn_roller=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシの鎮圧（5月中+下旬）
model.number_of_unit_corn_herbicide1=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシ除草剤散布（6月上旬）
model.number_of_unit_corn_herbicide2=Var(initialize=1,within=NonNegativeIntegers) #トウモロコシ除草剤散布（6月下旬）
model.number_of_unit_corn_harvest1=Var(initialize=0,within=NonNegativeIntegers) #トウモロコシの自家労働収穫（9月下旬）
model.number_of_unit_corn_harvest2=Var(initialize=0,within=NonNegativeIntegers) #トウモロコシの自家労働収穫（10月上旬）
model.number_of_unit_corn_harvest_contract1=Var(initialize=0,within=NonNegativeIntegers) #トウモロコシのコントラ委託収穫（9月下旬）
model.number_of_unit_corn_harvest_contract2=Var(initialize=0,within=NonNegativeIntegers) #トウモロコシのコントラ委託収穫（10月上旬）

#飼料生産部門_変数設定：組作業1ユニット当たりの各作業時間と1日当たり労働時間
model.hour_per_unit_italian_reap=Var(initialize=1,within=NonNegativeReals) #イタリアンの刈取（4月中+下旬）
model.hour_per_unit_italian_reverse=Var(initialize=1,within=NonNegativeReals) #イタリアンの反転（4月中+下旬）
model.hour_per_unit_italian_gather=Var(initialize=1,within=NonNegativeReals) #イタリアンの集草（4月中+下旬）
model.hour_per_unit_italian_roll=Var(initialize=1,within=NonNegativeReals) #イタリアンのロール成形（4月中+下旬）
model.hour_per_unit_italian_rapping=Var(initialize=1,within=NonNegativeReals) #イタリアンのラッピング（4月中+下旬）
model.hour_per_unit_italian_plant1=Var(initialize=1,within=NonNegativeReals) #イタリアンの播種（10月上旬）
model.hour_per_unit_italian_plant2=Var(initialize=1,within=NonNegativeReals) #イタリアンの播種（10月中旬）
model.hour_per_unit_italian_roller1=Var(initialize=1,within=NonNegativeReals) #イタリアンの鎮圧（10月上旬）
model.hour_per_unit_italian_roller2=Var(initialize=1,within=NonNegativeReals) #イタリアンの鎮圧（10月中旬）
model.hour_per_unit_rice_silage_reap=Var(initialize=1,within=NonNegativeReals) #飼料用稲の刈取（7月下旬+8月上旬+9月上旬）
model.hour_per_unit_rice_silage_reverse=Var(initialize=1,within=NonNegativeReals) #飼料用稲の反転（7月下旬+8月上旬+9月上旬）
model.hour_per_unit_rice_silage_gather=Var(initialize=1,within=NonNegativeReals) #飼料用稲の集草（7月下旬+8月上旬+9月上旬）
model.hour_per_unit_rice_silage_roll=Var(initialize=1,within=NonNegativeReals) #飼料用稲のロール成形（7月下旬+8月上旬+9月上旬）
model.hour_per_unit_rice_silage_rapping=Var(initialize=1,within=NonNegativeReals) #飼料用稲のラッピング（7月下旬+8月上旬+9月上旬）
model.hour_per_unit_corn_compost_spread=Var(initialize=1,within=NonNegativeReals) # トウモロコシの堆肥散布（5月中+下旬）
model.hour_per_unit_corn_plow=Var(initialize=1,within=NonNegativeReals) #トウモロコシのプラウ（5月中+下旬）
model.hour_per_unit_corn_rotary=Var(initialize=1,within=NonNegativeReals) #トウモロコシのロータリー（5月中+下旬）
model.hour_per_unit_corn_plant=Var(initialize=1,within=NonNegativeReals) #トウモロコシの播種（5月中+下旬）
model.hour_per_unit_corn_roller=Var(initialize=1,within=NonNegativeReals) #トウモロコシの鎮圧（5月中+下旬）
model.hour_per_unit_corn_herbicide1=Var(initialize=1,within=NonNegativeReals) #トウモロコシ除草剤散布（6月上旬）
model.hour_per_unit_corn_herbicide2=Var(initialize=1,within=NonNegativeReals) #トウモロコシ除草剤散布（6月下旬）
model.hour_per_unit_corn_harvest1=Var(initialize=1,within=NonNegativeReals) #トウモロコシの自家労働収穫（9月下旬）
model.hour_per_unit_corn_harvest2=Var(initialize=1,within=NonNegativeReals) #トウモロコシの自家労働収穫（10月上旬）
model.hour_per_unit_corn_harvest_contract1=Var(initialize=1,within=NonNegativeReals) #トウモロコシのコントラ委託収穫（9月下旬）
model.hour_per_unit_corn_harvest_contract2=Var(initialize=1,within=NonNegativeReals) #トウモロコシのコントラ委託収穫（10月上旬）
model.day_hour_fodder=Var(initialize=6,within=NonNegativeReals) #飼料生産に係る1日当たり労働時間

#飼料生産部門_変数設定：飼料作物の面積
model.area_italian=Var(initialize=1500,within=NonNegativeReals) #イタリアンの生産面積
model.area_italian1=Var(initialize=500,within=NonNegativeReals) #イタリアンの播種面積（10月上旬）
model.area_italian2=Var(initialize=500,within=NonNegativeReals) #イタリアンの播種面積（10月中旬）
model.area_rice_silage=Var(initialize=4000,within=NonNegativeIntegers) #飼料稲の生産面積
model.area_corn_silage=Var(initialize=1000,within=NonNegativeReals) #トウモロコシの生産面積
model.area_contract_corn_herbicide1=Var(initialize=0,within=NonNegativeReals) #トウモロコシ除草剤散布の委託面積（6月上旬）
model.area_contract_corn_herbicide2=Var(initialize=0,within=NonNegativeReals) #トウモロコシ除草剤散布の委託面積（6月下旬）
model.area_corn_harvest_self1=Var(initialize=1000,within=NonNegativeReals) #トウモロコシの自家労働収穫面積（9月下旬）
model.area_corn_harvest_self2=Var(initialize=1000,within=NonNegativeReals) #トウモロコシの自家労働収穫面積（10月上旬）
model.area_contract_corn_harvest=Var(initialize=0,within=NonNegativeReals) #トウモロコシのコントラ委託収穫面積合計
model.area_contract_corn_harvest1=Var(initialize=0,within=NonNegativeReals) #トウモロコシのコントラ委託収穫面積（9月下旬）
model.area_contract_corn_harvest2=Var(initialize=0,within=NonNegativeReals) #トウモロコシのコントラ委託収穫面積（10月上旬）

# model.simu_riceWCS=Constraint(expr=model.area_rice_silage<=4000) #a, 飼料稲の面積, 指定しないときはコメントアウト


#飼料生産部門_変数設定：臨時雇用の人数
model.number_of_corn_silage_day_labor_self1=Var(initialize=1,within=NonNegativeIntegers) #臨時雇用：自家収穫（9月下旬）
model.number_of_corn_silage_day_labor_self2=Var(initialize=1,within=NonNegativeIntegers) #臨時雇用：自家収穫（10月上旬）
model.number_of_corn_silage_day_labor_contract1=Var(initialize=1,within=NonNegativeIntegers) #臨時雇用：コントラクタ（9月下旬）
model.number_of_corn_silage_day_labor_contract2=Var(initialize=1,within=NonNegativeIntegers) #臨時雇用：コントラクタ（10月上旬）

#飼料生産部門_制約：組作業ユニット稼働時間の下限（１時間~）と上限（~旬別労働可能時間）
##トウモロコシ
model.const_min_unit_hour_corn_compost=Constraint(expr=model.hour_per_unit_corn_compost_spread>=1) 
model.const_min_unit_hour_corn_plow=Constraint(expr=model.hour_per_unit_corn_plow>=1)
model.const_min_unit_hour_corn_rotary=Constraint(expr=model.hour_per_unit_corn_rotary>=1)
model.const_min_unit_hour_corn_plant=Constraint(expr=model.hour_per_unit_corn_plant>=1)
model.const_min_unit_hour_corn_roller=Constraint(expr=model.hour_per_unit_corn_roller>=1)
model.const_min_unit_hour_corn_herbicide1=Constraint(expr=model.hour_per_unit_corn_herbicide1>=1)
model.const_min_unit_hour_corn_herbicide2=Constraint(expr=model.hour_per_unit_corn_herbicide2>=1)
model.const_min_unit_hour_corn_harvest1=Constraint(expr=model.hour_per_unit_corn_harvest1>=1)
model.const_min_unit_hour_corn_harvest2=Constraint(expr=model.hour_per_unit_corn_harvest2>=1)
model.const_min_unit_hour_corn_harvest_contract1=Constraint(expr=model.hour_per_unit_corn_harvest_contract1>=1)
model.const_min_unit_hour_corn_harvest_contract2=Constraint(expr=model.hour_per_unit_corn_harvest_contract2>=1)
model.const_max_unit_hour_corn_harvest1=Constraint(expr=
                                                   model.hour_per_unit_corn_harvest1<=10*sunny_rate*model.day_hour_fodder)
model.const_max_unit_hour_corn_harvest2=Constraint(expr=
                                                   model.hour_per_unit_corn_harvest2<=10*sunny_rate*model.day_hour_fodder)
model.const_max_unit_hour_corn_harvest_contract1=Constraint(expr=
                                                            model.hour_per_unit_corn_harvest_contract1<=10*sunny_rate*model.day_hour_fodder)
model.const_max_unit_hour_corn_harvest_contract2=Constraint(expr=
                                                            model.hour_per_unit_corn_harvest_contract2<=10*sunny_rate*model.day_hour_fodder)
##イタリアン
model.const_min_unit_hour_italian_plant1=Constraint(expr=model.hour_per_unit_italian_plant1>=1)
model.const_min_unit_hour_italian_plant2=Constraint(expr=model.hour_per_unit_italian_plant2>=1)
model.const_min_unit_hour_italian_roller1=Constraint(expr=model.hour_per_unit_italian_roller1>=1)
model.const_min_unit_hour_italian_roller2=Constraint(expr=model.hour_per_unit_italian_roller2>=1)
model.const_min_unit_hour_italian_reap=Constraint(expr=model.hour_per_unit_italian_reap>=1)
model.const_min_unit_hour_italian_reverse=Constraint(expr=model.hour_per_unit_italian_reverse>=1)
model.const_min_unit_hour_italian_gather=Constraint(expr=model.hour_per_unit_italian_gather>=1)
model.const_min_unit_hour_italian_roll=Constraint(expr=model.hour_per_unit_italian_roll>=1)
model.const_min_unit_hour_italian_rapping=Constraint(expr=model.hour_per_unit_italian_rapping>=1)
model.const_max_unit_hour_ita1=Constraint(expr=model.hour_per_unit_italian_plant1<=10*sunny_rate*model.day_hour_fodder)
model.const_max_unit_hour_ita2=Constraint(expr=model.hour_per_unit_italian_roller1<=10*sunny_rate*model.day_hour_fodder)
model.const_max_unit_hour_ita3=Constraint(expr=model.hour_per_unit_italian_plant2<=10*sunny_rate*model.day_hour_fodder)
model.const_max_unit_hour_ita4=Constraint(expr=model.hour_per_unit_italian_roller2<=10*sunny_rate*model.day_hour_fodder)
##稲WCS
model.const_min_unit_hour_rice_silage_reap=Constraint(expr=model.hour_per_unit_rice_silage_reap>=1)
model.const_min_unit_hour_rice_silage_reverse=Constraint(expr=model.hour_per_unit_rice_silage_reverse>=1)
model.const_min_unit_hour_rice_silage_gather=Constraint(expr=model.hour_per_unit_rice_silage_gather>=1)
model.const_min_unit_hour_rice_silage_roll=Constraint(expr=model.hour_per_unit_rice_silage_roll>=1)
model.const_min_unit_hour_rice_silage_rapping=Constraint(expr=model.hour_per_unit_rice_silage_rapping>=1)

#飼料生産部門_制約：組作業ユニット数の上限
## トウモロコシ
model.const_number_of_unit_corn_compost_spread=Constraint(expr=model.number_of_unit_corn_compost_spread<=3)
model.const_number_of_unit_corn_plow=Constraint(expr=model.number_of_unit_corn_plow<=3)
model.const_number_of_unit_corn_rotary=Constraint(expr=model.number_of_unit_corn_rotary<=3)
model.const_number_of_unit_corn_plant=Constraint(expr=model.number_of_unit_corn_plant<=3)
model.const_number_of_unit_corn_roller=Constraint(expr=model.number_of_unit_corn_roller<=3)
model.const_number_of_unit_corn_herbicide1=Constraint(expr=model.number_of_unit_corn_herbicide1<=3)
model.const_number_of_unit_corn_herbicide2=Constraint(expr=model.number_of_unit_corn_herbicide2<=3)
model.const_number_of_unit_corn_harvest1=Constraint(expr=model.number_of_unit_corn_harvest1<=1)
model.const_number_of_unit_corn_harvest2=Constraint(expr=model.number_of_unit_corn_harvest2<=1)
model.const_number_of_unit_corn_contract_harvest1=Constraint(expr=model.number_of_unit_corn_harvest_contract1<=1)
model.const_number_of_unit_corn_contract_harvest2=Constraint(expr=model.number_of_unit_corn_harvest_contract2<=1)
##イタリアン
# model.const_number_of_unit_italian_plant1=Constraint(expr=model.number_of_unit_italian_plant1<=3)
# model.const_number_of_unit_italian_roller1=Constraint(expr=model.number_of_unit_italian_roller1<=3)
# model.const_number_of_unit_italian_plant2=Constraint(expr=model.number_of_unit_italian_plant2<=3)
# model.const_number_of_unit_italian_roller2=Constraint(expr=model.number_of_unit_italian_roller2<=3)
# model.const_number_of_unit_italian_reap=Constraint(expr=model.number_of_unit_italian_reap<=3)
# model.const_number_of_unit_italian_reverse=Constraint(expr=model.number_of_unit_italian_reverse<=3)
# model.const_number_of_unit_italian_gather=Constraint(expr=model.number_of_unit_italian_gather<=3)
# model.const_number_of_unit_italian_roll=Constraint(expr=model.number_of_unit_italian_roll<=3)
# model.const_number_of_unit_italian_rapping=Constraint(expr=model.number_of_unit_italian_rapping<=3)
##稲WCS
model.const_number_of_unit_rice_silage_reap=Constraint(expr=model.number_of_unit_rice_silage_reap<=3)
model.const_number_of_unit_rice_silage_reverse=Constraint(expr=model.number_of_unit_rice_silage_reverse<=3)
model.const_number_of_unit_rice_silage_gather=Constraint(expr=model.number_of_unit_rice_silage_gather<=3)
model.const_number_of_unit_rice_silage_roll=Constraint(expr=model.number_of_unit_rice_silage_roll<=3)
model.const_number_of_unit_rice_silage_rapping=Constraint(expr=model.number_of_unit_rice_silage_rapping<=3)

#飼料生産部門_制約：作付可能な面積は経営耕地面積以下
model.const_cropping_area_corn=Constraint(expr=model.area_corn_silage<=farming_area)
model.const_cropping_area_italian=Constraint(expr=model.area_italian<=farming_area+model.area_rice_silage)

#飼料生産部門_制約：各工程の作業面積と生産面積は等しい（イタリアン）
model.const_area_italian=Constraint(expr=model.area_italian==model.area_italian1+model.area_italian2)
model.const_process_area_italian_plant1=Constraint(expr=
                                                  model.number_of_unit_italian_plant1*model.hour_per_unit_italian_plant1
                                                   *operation_ef_grass_rotary_seed
                                                  ==model.area_italian1)
model.const_process_area_italian_plant2=Constraint(expr=
                                                  model.number_of_unit_italian_plant2*model.hour_per_unit_italian_plant2
                                                   *operation_ef_grass_rotary_seed
                                                  ==model.area_italian2)
model.const_process_area_italian_roller1=Constraint(expr=
                                                   model.number_of_unit_italian_roller1*model.hour_per_unit_italian_roller1
                                                    *operation_ef_grass_press
                                                   ==model.area_italian1)
model.const_process_area_italian_roller2=Constraint(expr=
                                                    model.number_of_unit_italian_roller2*model.hour_per_unit_italian_roller2
                                                    *operation_ef_grass_press
                                                   ==model.area_italian2)
model.const_process_area_italian_reap=Constraint(expr=
                                                 model.number_of_unit_italian_reap*model.hour_per_unit_italian_reap
                                                 *operation_ef_grass_reap
                                                 ==model.area_italian)
model.const_process_area_italian_reverse=Constraint(expr=
                                                    model.number_of_unit_italian_reverse*model.hour_per_unit_italian_reverse
                                                    *operation_ef_grass_reverse
                                                    ==model.area_italian)
model.const_process_area_italian_gather=Constraint(expr=
                                                   model.number_of_unit_italian_gather*model.hour_per_unit_italian_gather
                                                   *operation_ef_grass_gather
                                                   ==model.area_italian)
model.const_process_area_italian_roll=Constraint(expr=
                                                 model.number_of_unit_italian_roll*model.hour_per_unit_italian_roll
                                                 *operation_ef_grass_role
                                                 ==model.area_italian)
model.const_process_area_italian_rapping=Constraint(expr=
                                                    model.number_of_unit_italian_rapping*model.hour_per_unit_italian_rapping
                                                    *operation_ef_grass_rapping
                                                    ==model.area_italian)

#飼料生産部門_制約：各工程の作業面積と生産面積は等しい（飼料用稲）
model.const_process_area_rice_silage_reap=Constraint(expr=
                                                     model.number_of_unit_rice_silage_reap*model.hour_per_unit_rice_silage_reap
                                                     *operation_ef_grass_reap
                                                     ==model.area_rice_silage)
model.const_process_area_rice_silage_reverse=Constraint(expr=
                                                        model.number_of_unit_rice_silage_reverse*model.hour_per_unit_rice_silage_reverse
                                                        *operation_ef_grass_reverse
                                                        ==model.area_rice_silage)
model.const_process_area_rice_silage_gather=Constraint(expr=
                                                       model.number_of_unit_rice_silage_gather*model.hour_per_unit_rice_silage_gather
                                                       *operation_ef_grass_gather
                                                       ==model.area_rice_silage)
model.const_process_area_rice_silage_roll=Constraint(expr=
                                                     model.number_of_unit_rice_silage_roll*model.hour_per_unit_rice_silage_roll
                                                     *operation_ef_grass_role
                                                     ==model.area_rice_silage)
model.const_process_area_rice_silage_rapping=Constraint(expr=
                                                        model.number_of_unit_rice_silage_rapping*model.hour_per_unit_rice_silage_rapping
                                                        *operation_ef_grass_rapping
                                                        ==model.area_rice_silage)

#飼料生産部門_制約：各工程の作業面積と生産面積は等しい（トウモロコシ）
model.const_process_area_corn_spread=Constraint(expr=
                                                model.number_of_unit_corn_compost_spread*model.hour_per_unit_corn_compost_spread
                                                *operation_ef_corn_compost_spread
                                                ==model.area_corn_silage)
model.const_process_area_corn_plow=Constraint(expr=
                                              model.number_of_unit_corn_plow*model.hour_per_unit_corn_plow
                                              *operation_ef_corn_plow
                                              ==model.area_corn_silage)
model.const_process_area_corn_rotary=Constraint(expr=
                                                model.number_of_unit_corn_rotary*model.hour_per_unit_corn_rotary
                                                *operation_ef_corn_rotary
                                                ==model.area_corn_silage)
model.const_process_area_corn_plant=Constraint(expr=
                                               model.number_of_unit_corn_plant*model.hour_per_unit_corn_plant
                                               *operation_ef_corn_plant
                                               ==model.area_corn_silage)
model.const_process_area_corn_roller=Constraint(expr=
                                                model.number_of_unit_corn_roller*model.hour_per_unit_corn_roller
                                                *operation_ef_corn_press
                                                ==model.area_corn_silage)
model.const_process_area_corn_herbicide1=Constraint(expr=
                                                    model.number_of_unit_corn_herbicide1*model.hour_per_unit_corn_herbicide1*operation_ef_corn_herbicide1
                                                    +model.area_contract_corn_herbicide1
                                                    ==model.area_corn_silage)
model.const_process_area_corn_herbicide2=Constraint(expr=
                                                    model.number_of_unit_corn_herbicide2*model.hour_per_unit_corn_herbicide2*operation_ef_corn_herbicide2
                                                    +model.area_contract_corn_herbicide2
                                                    ==model.area_corn_silage)
model.const_process_area_corn_harvest=Constraint(expr= #自家収穫面積+コントラ委託面積＝トウモロコシ生産面積
                                                 model.area_corn_harvest_self1+model.area_corn_harvest_self2+model.area_contract_corn_harvest
                                                 ==model.area_corn_silage) 
model.const_process_area_corn_self1=Constraint(expr= #自家収穫（9月下旬）：作業面積=収穫面積
                                               model.number_of_unit_corn_harvest1*model.hour_per_unit_corn_harvest1*operation_ef_corn_harvest3
                                               ==model.area_corn_harvest_self1) 
model.const_process_area_corn_self2=Constraint(expr= #自家収穫（10月上旬）：作業面積=収穫面積
                                                 model.number_of_unit_corn_harvest2*model.hour_per_unit_corn_harvest2*operation_ef_corn_harvest3
                                                 ==model.area_corn_harvest_self2) 
model.const_process_area_corn_contra1=Constraint(expr= #コントラ委託（9月下旬）：作業面積=収穫面積
                                             model.number_of_unit_corn_harvest_contract1
                                             *model.hour_per_unit_corn_harvest_contract1*operation_ef_corn_harvest
                                             ==model.area_contract_corn_harvest1)
model.const_process_area_corn_contra2=Constraint(expr= #コントラ委託（10月上旬）：作業面積=収穫面積
                                             model.number_of_unit_corn_harvest_contract2
                                             *model.hour_per_unit_corn_harvest_contract2*operation_ef_corn_harvest
                                             ==model.area_contract_corn_harvest2)
model.const_process_area_corn_contra=Constraint(expr= #コントラ委託（9月下旬）+コントラ委託（10月上旬）=コントラ委託面積
                                                model.area_contract_corn_harvest1+model.area_contract_corn_harvest2
                                                ==model.area_contract_corn_harvest)










#飼料設計部門_前提:搾乳牛の要求養分
def calc_requirements_milk(milk,fat,feed_type,birth_count,delivery_day,w):
    fcm=(15*fat/100+0.4)*milk #脂肪補正乳量

    #CFA(養分量補正係数)
    if feed_type==1:
        cfa=1+milk/15*0.04
    else:
        cfa=1+milk/15*0.035

    #泌乳初期dmi補正
    if birth_count>=2:
        dmi_correct=1-0.3531*math.exp(-0.3247*round(delivery_day/7))
    else:
        dmi_correct=0.6446*(round(delivery_day/7)**0.1752)
        
    #DMI推定
    if round(delivery_day/7)>=11:
        if birth_count>=2:
            dmi=1.3922+0.05839*(w**0.75)+0.40497*fcm
        else:
            dmi=1.912+0.07031*(w**0.75)+0.34923*fcm
    else:
        if birth_count>=2:
            dmi=1.3922+0.05839*(w**0.75)+0.40497*fcm*dmi_correct
        else:
            dmi=1.912+0.07031*(w**0.75)+0.34923*fcm*dmi_correct
        
    #CP要求量計算
    dcpr1=2.71*(w**0.75); cpr1=dcpr1/0.6 #維持    
    dcpr2=(26.6+5.3*fat)*milk; cpr2=dcpr2/0.65 #産乳    
    cpr=cpr1+cpr2 #維持＋産乳

    #エネルギー要求量  
    merc1=0.1163*(w**0.75); der1=merc1/0.82; tdnr1=der1/4.41 #維持   
    gemc=(0.0913*fat+0.3678)*milk; merc2=gemc/0.62; der2=merc2/0.82; tdnr2=der2/4.41 #産乳
    tdnr=tdnr1+tdnr2 #維持＋産乳
    tdnr=tdnr*cfa; cpr=cpr*cfa #乾物摂取量の増加に伴う補正（cfa）
    
    return dmi,tdnr,cpr

#飼料設計部門_前提: その他乳牛の要求養分
dmi_menue_raised=6.62; tdnr_menue_raised=4.75; cpr_menue_raised=783 #育成牛メニュー（300日）
dmi_menue_dry=11.1; tdnr_menue_dry=4.14; cpr_menue_dry=581#乾乳牛メニュー（650kg）

#飼料設計部門_前提: 各飼料の価格と栄養分（栄養分は乾物あたり, 価格は原物kgあたり円）
corn_silage_dmr=0.272; corn_silage_tdn=0.679; corn_silage_cp=0.085 #トウモロコシサイレージ
italian_silage_cost=23; italian_silage_dmr=0.3; italian_silage_tdn=0.692; italian_silage_cp=0.137 #イタリアンサイレージ
fodder_rice_silage_cost=15; fodder_rice_silage_dmr=0.3; fodder_rice_silage_tdn=0.479; fodder_rice_silage_cp=0.1 #飼料稲
timothy_cost=69; timothy_dmr=0.85; timothy_tdn=0.626; timothy_cp=0.101 #購入チモシー
alfalfa_cost=57; alfalfa_dmr=0.85; alfalfa_tdn=0.577; alfalfa_cp=0.191 #購入アルファルファ
mixed_cost=61.4; mixed_dmr=0.9; mixed_tdn=0.88; mixed_cp=0.2 #配合飼料
soybean_meal_cost=76; soybean_meal_dmr=0.9; soybean_meal_tdn=0.908; soybean_meal_cp=0.502; soybean_meal_cat=0 #大豆粕
flakedcorn_cost=38; flakedcorn_dmr=0.85; flakedcorn_tdn=0.936; flakedcorn_cp=0.088 #圧ぺんトウモロコシ

dmi_menue,tdnr_menue,cpr_menue=calc_requirements_milk(milk,fat,feed_type,birth_count,delivery_day,w)
print (dmi_menue,tdnr_menue,cpr_menue)

#飼料設計部門_変数設定：搾乳牛用メニュー
model.dmi=Var(initialize=20,within=NonNegativeReals)
model.fodder_rate=Var(initialize=10,within=NonNegativeReals)
model.corn_silage_menue=Var(initialize=15,within=NonNegativeReals)
model.italian_silage_menue=Var(initialize=10,within=NonNegativeReals)
model.fodder_rice_silage_menue=Var(initialize=10,within=NonNegativeReals)
model.timothy_menue=Var(initialize=0,within=NonNegativeReals)
model.alfalfa_menue=Var(initialize=0,within=NonNegativeReals)
model.mixed_menue=Var(initialize=5,within=NonNegativeReals)
model.soybean_menue=Var(initialize=1,within=NonNegativeReals)
model.flakedcorn_menue=Var(initialize=5,within=NonNegativeReals)
#飼料設計部門_変数設定:育成牛用メニュー
model.italian_silage_menue_raised=Var(initialize=1,within=NonNegativeReals)
model.fodder_rice_silage_menue_raised=Var(initialize=10,within=NonNegativeReals)
model.timothy_menue_raised=Var(initialize=1,within=NonNegativeReals)
model.alfalfa_menue_raised=Var(initialize=1,within=NonNegativeReals)
model.soybean_menue_raised=Var(initialize=1,within=NonNegativeReals)
model.flakedcorn_menue_raised=Var(initialize=1,within=NonNegativeReals)
mixed_menue_raised=0
#飼料設計部門_変数設定:乾乳牛用メニュー
model.italian_silage_menue_dry=Var(initialize=1,within=NonNegativeReals)
model.fodder_rice_silage_menue_dry=Var(initialize=20,within=NonNegativeReals)
model.timothy_menue_dry=Var(initialize=1,within=NonNegativeReals)
model.alfalfa_menue_dry=Var(initialize=1,within=NonNegativeReals)
model.mixed_menue_dry=Var(initialize=1,within=NonNegativeReals)
model.soybean_menue_dry=Var(initialize=1,within=NonNegativeReals)
model.flakedcorn_menue_dry=Var(initialize=1,within=NonNegativeReals)

#飼料設計部門_制約：搾乳牛養分要求量
model.const_dmi_lower=Constraint(expr=
                                 model.corn_silage_menue*corn_silage_dmr
                                 +model.italian_silage_menue*italian_silage_dmr
                                 +model.fodder_rice_silage_menue*fodder_rice_silage_dmr
                                 +model.timothy_menue*timothy_dmr
                                 +model.alfalfa_menue*alfalfa_dmr
                                 +model.mixed_menue*mixed_dmr
                                 +model.soybean_menue*soybean_meal_dmr
                                 +model.flakedcorn_menue*flakedcorn_dmr
                                 >=dmi_menue*0.95)
model.const_dmi_upper=Constraint(expr=
                                 model.corn_silage_menue*corn_silage_dmr
                                 +model.italian_silage_menue*italian_silage_dmr
                                 +model.fodder_rice_silage_menue*fodder_rice_silage_dmr
                                 +model.timothy_menue*timothy_dmr
                                 +model.alfalfa_menue*alfalfa_dmr
                                 +model.mixed_menue*mixed_dmr
                                 +model.soybean_menue*soybean_meal_dmr
                                 +model.flakedcorn_menue*flakedcorn_dmr
                                 <=dmi_menue*1.05)
model.const_tdn_lower=Constraint(expr=
                                 model.corn_silage_menue*corn_silage_dmr*corn_silage_tdn
                                 +model.italian_silage_menue*italian_silage_dmr*italian_silage_tdn
                                 +model.fodder_rice_silage_menue*fodder_rice_silage_dmr*fodder_rice_silage_tdn
                                 +model.timothy_menue*timothy_dmr*timothy_tdn
                                 +model.alfalfa_menue*alfalfa_dmr*alfalfa_tdn
                                 +model.mixed_menue*mixed_dmr*mixed_tdn
                                 +model.soybean_menue*soybean_meal_dmr*soybean_meal_tdn
                                 +model.flakedcorn_menue*flakedcorn_dmr*flakedcorn_tdn
                                 >=tdnr_menue)
model.const_tdn_upper=Constraint(expr=model.corn_silage_menue*corn_silage_dmr*corn_silage_tdn
                                 +model.italian_silage_menue*italian_silage_dmr*italian_silage_tdn
                                 +model.fodder_rice_silage_menue*fodder_rice_silage_dmr*fodder_rice_silage_tdn
                                 +model.timothy_menue*timothy_dmr*timothy_tdn
                                 +model.alfalfa_menue*alfalfa_dmr*alfalfa_tdn
                                 +model.mixed_menue*mixed_dmr*mixed_tdn
                                 +model.soybean_menue*soybean_meal_dmr*soybean_meal_tdn
                                 +model.flakedcorn_menue*flakedcorn_dmr*flakedcorn_tdn
                                 <=tdnr_menue*1.1)
model.const_cpn_lower=Constraint(expr=
                                 model.corn_silage_menue*corn_silage_dmr*corn_silage_cp
                                 +model.italian_silage_menue*italian_silage_dmr*italian_silage_cp
                                 +model.fodder_rice_silage_menue*fodder_rice_silage_dmr*fodder_rice_silage_cp
                                 +model.timothy_menue*timothy_dmr*timothy_cp
                                 +model.alfalfa_menue*alfalfa_dmr*alfalfa_cp
                                 +model.mixed_menue*mixed_dmr*mixed_cp
                                 +model.soybean_menue*soybean_meal_dmr*soybean_meal_cp
                                 +model.flakedcorn_menue*flakedcorn_dmr*flakedcorn_cp
                                 >=cpr_menue/1000)
model.const_cpn_upper=Constraint(expr=
                                model.corn_silage_menue*corn_silage_dmr*corn_silage_cp
                                +model.italian_silage_menue*italian_silage_dmr*italian_silage_cp
                                +model.fodder_rice_silage_menue*fodder_rice_silage_dmr*fodder_rice_silage_cp
                                +model.timothy_menue*timothy_dmr*timothy_cp
                                +model.alfalfa_menue*alfalfa_dmr*alfalfa_cp
                                +model.mixed_menue*mixed_dmr*mixed_cp
                                +model.soybean_menue*soybean_meal_dmr*soybean_meal_cp
                                +model.flakedcorn_menue*flakedcorn_dmr*flakedcorn_cp
                                <=cpr_menue/1000*1.1)
model.const_fodder_rate1=Constraint(expr=
                                    model.dmi
                                    ==model.corn_silage_menue*corn_silage_dmr
                                    +model.italian_silage_menue*italian_silage_dmr
                                    +model.fodder_rice_silage_menue*fodder_rice_silage_dmr
                                    +model.timothy_menue*timothy_dmr
                                    +model.alfalfa_menue*alfalfa_dmr
                                    +model.mixed_menue*mixed_dmr
                                    +model.soybean_menue*soybean_meal_dmr
                                    +model.flakedcorn_menue*flakedcorn_dmr)
model.const_fodder_rate2=Constraint(expr=
                                    model.fodder_rate
                                    ==model.corn_silage_menue*corn_silage_dmr
                                    +model.italian_silage_menue*italian_silage_dmr
                                    +model.fodder_rice_silage_menue*fodder_rice_silage_dmr
                                    +model.timothy_menue*timothy_dmr
                                    +model.alfalfa_menue*alfalfa_dmr)

#飼料設計部門_制約：乾乳牛養分要求量
model.const_dmi_lower_dry=Constraint(expr=
                                     model.italian_silage_menue_dry*italian_silage_dmr
                                     +model.fodder_rice_silage_menue_dry*fodder_rice_silage_dmr
                                     +model.timothy_menue_dry*timothy_dmr
                                     +model.alfalfa_menue_dry*alfalfa_dmr
                                     +model.mixed_menue_dry*mixed_dmr
                                     +model.soybean_menue_dry*soybean_meal_dmr
                                     +model.flakedcorn_menue_dry*flakedcorn_dmr
                                     >=dmi_menue_dry*0.95)
model.const_dmi_upper_dry=Constraint(expr=
                                     model.italian_silage_menue_dry*italian_silage_dmr
                                     +model.fodder_rice_silage_menue_dry*fodder_rice_silage_dmr
                                     +model.timothy_menue_dry*timothy_dmr
                                     +model.alfalfa_menue_dry*alfalfa_dmr
                                     +model.mixed_menue_dry*mixed_dmr
                                     +model.soybean_menue_dry*soybean_meal_dmr
                                     +model.flakedcorn_menue_dry*flakedcorn_dmr
                                     <=dmi_menue_dry*1.05)
model.const_tdn_lower_dry=Constraint(expr=
                                     model.italian_silage_menue_dry*italian_silage_dmr*italian_silage_tdn
                                     +model.fodder_rice_silage_menue_dry*fodder_rice_silage_dmr*fodder_rice_silage_tdn
                                     +model.timothy_menue_dry*timothy_dmr*timothy_tdn
                                     +model.alfalfa_menue_dry*alfalfa_dmr*alfalfa_tdn
                                     +model.mixed_menue_dry*mixed_dmr*mixed_tdn
                                     +model.soybean_menue_dry*soybean_meal_dmr*soybean_meal_tdn
                                     +model.flakedcorn_menue_dry*flakedcorn_dmr*flakedcorn_tdn
                                     >=tdnr_menue_dry)
model.const_cpn_lower_dry=Constraint(expr=
                                     model.italian_silage_menue_dry*italian_silage_dmr*italian_silage_cp
                                     +model.fodder_rice_silage_menue_dry*fodder_rice_silage_dmr*fodder_rice_silage_cp
                                     +model.timothy_menue_dry*timothy_dmr*timothy_cp
                                     +model.alfalfa_menue_dry*alfalfa_dmr*alfalfa_cp
                                     +model.mixed_menue_dry*mixed_dmr*mixed_cp
                                     +model.soybean_menue_dry*soybean_meal_dmr*soybean_meal_cp
                                     +model.flakedcorn_menue_dry*flakedcorn_dmr*flakedcorn_cp
                                     >=cpr_menue_dry/1000)

#飼料設計部門_制約：育成牛養分要求量
model.const_dmi_lower_raised=Constraint(expr=
                                        model.italian_silage_menue_raised*italian_silage_dmr
                                        +model.fodder_rice_silage_menue_raised*fodder_rice_silage_dmr
                                        +model.timothy_menue_raised*timothy_dmr
                                        +model.alfalfa_menue_raised*alfalfa_dmr
                                        +model.soybean_menue_raised*soybean_meal_dmr
                                        +model.flakedcorn_menue_raised*flakedcorn_dmr                                    
                                        >=dmi_menue_raised*0.95)
model.const_dmi_upper_raised=Constraint(expr=
                                        model.italian_silage_menue_raised*italian_silage_dmr
                                        +model.fodder_rice_silage_menue_raised*fodder_rice_silage_dmr
                                        +model.timothy_menue_raised*timothy_dmr
                                        +model.alfalfa_menue_raised*alfalfa_dmr
                                        +model.soybean_menue_raised*soybean_meal_dmr
                                        +model.flakedcorn_menue_raised*flakedcorn_dmr
                                        <=dmi_menue_raised*1.05)
model.const_tdn_lower_raised=Constraint(expr=
                                        model.italian_silage_menue_raised*italian_silage_dmr*italian_silage_tdn
                                        +model.fodder_rice_silage_menue_raised*fodder_rice_silage_dmr*fodder_rice_silage_tdn
                                        +model.timothy_menue_raised*timothy_dmr*timothy_tdn
                                        +model.alfalfa_menue_raised*alfalfa_dmr*alfalfa_tdn
                                        +model.soybean_menue_raised*soybean_meal_dmr*soybean_meal_tdn
                                        +model.flakedcorn_menue_raised*flakedcorn_dmr*flakedcorn_tdn
                                        >=tdnr_menue_raised)
model.const_cpn_lower_raised=Constraint(expr=
                                        model.italian_silage_menue_raised*italian_silage_dmr*italian_silage_cp
                                        +model.fodder_rice_silage_menue_raised*fodder_rice_silage_dmr*fodder_rice_silage_cp
                                        +model.timothy_menue_raised*timothy_dmr*timothy_cp
                                        +model.alfalfa_menue_raised*alfalfa_dmr*alfalfa_cp
                                        +model.soybean_menue_raised*soybean_meal_dmr*soybean_meal_cp
                                        +model.flakedcorn_menue_raised*flakedcorn_dmr*flakedcorn_cp
                                        >=cpr_menue_raised/1000)

#飼料設計部門_制約：給与可能な量は生産量以下, 粗濃比（粗飼料割合は50%以上）
model.const_corn_amount=Constraint(expr=
                                   model.corn_silage_menue*model.cattle_milking*365
                                   <=model.area_corn_silage*yield_corn)
model.const_italian_amount=Constraint(expr=
                                      model.italian_silage_menue*model.cattle_milking*365
                                      +model.italian_silage_menue_raised*model.cattle_raised_total*365
                                      +model.italian_silage_menue_dry*model.cattle_dry*365
                                     <=model.area_italian*yield_italian)
model.cost_fodder_rice=Constraint(expr=
                                  model.fodder_rice_silage_menue*model.cattle_milking*365
                                  +model.fodder_rice_silage_menue_raised*model.cattle_raised_total*365
                                  +model.fodder_rice_silage_menue_dry*model.cattle_dry*365
                                 <=model.area_rice_silage*yield_fodder_rice)
model.const_fodder_rate=Constraint(expr=model.fodder_rate/model.dmi>=0.5) #粗飼料割合は50%以上


#飼料設計部門_制約：搾乳牛給与メニュー
model.const_corn_silage=Constraint(expr=model.corn_silage_menue<=30) #トウモロコシサイレージは30kg以下
model.const_italian_silage=Constraint(expr=model.italian_silage_menue<=10) #イタリアンサイレージは10kg以下
model.const_fodder_rice_silage=Constraint(expr=model.fodder_rice_silage_menue<=10) #飼料稲サイレージは10kg以下
model.const_alfalfa=Constraint(expr=model.alfalfa_menue<=3) #乾草アルファルファは3kg以下
model.const_mix=Constraint(expr=model.mixed_menue>=5) #配合飼料は5kg以上

#飼料設計部門_制約：乾乳牛メニュー
model.const_concentrate_dry=Constraint(expr= #配合, 大豆粕, 圧ぺんトウモロコシ合わせて3kg以下
                                       model.mixed_menue_dry+model.soybean_menue_dry+model.flakedcorn_menue_dry<=3)
model.const_alfalf_dry=Constraint(expr=model.alfalfa_menue_dry<=3) #購入乾草アルファルファは3kg以下

#飼料設計部門_制約：育成牛メニュー
model.const_concentrate_raised=Constraint(expr= #大豆粕, 圧ぺんトウモロコシ合わせて2kg以下
                                          model.soybean_menue_raised+model.flakedcorn_menue_raised<=2)
model.const_alfalf_raised=Constraint(expr=model.alfalfa_menue_raised<=3) #購入乾草アルファルファは3kg以下










#機械調達部門_前提：固定費用（機械施設年償却費）
cost_of_machine_tractor_130ps=13716*1000/7 #ニューホランドT6.160
cost_of_machine_tractor_160ps=16848*1000/7 #ニューホランドT7.210
cost_of_machine_chemper_head3=800*10000/7 #3条のケンパーヘッド 
cost_of_machine_dump=400*10000/7 #ダンプ
cost_of_machine_wheel_loader_bucket=500*10000/7 #ローダーバケット，牛舎で使用
cost_of_machine_wheel_loader_grab=500*10000/7 #ローダーグラブ，牛舎で使用
cost_of_machine_compost_spreader=300*10000/7  #65〜80psトラクタ対応
cost_of_machine_plow=135*10000/7  #R143HBB
cost_of_machine_corn_planter=320*10000/7 #タカキタJS4127
cost_of_machine_pressure_roller=250*10000/7 #鎮圧ローラー
cost_of_machine_passenger=313*10000/7 #乗用管理機RVH500G/90N-18
cost_of_machine_rotary=170*10000/7 #ロータリー
cost_of_machine_rotary_seeder=80*10000/7 #播種機（イタリアンに使用）
cost_of_machine_more=(394.2+316.44)*10000/7 #エムエスケーFC313F（フロント）+エムエスケーFC283（直装）
cost_of_machine_tedder=390*10000/7 #テッダー
cost_of_machine_rake=360*10000/7 #レーキ
cost_of_machine_role_baler=650*10000/7 #ロールベーラー
cost_of_machine_rapper=246*10000/7 #ラッピングマシン
cost_of_silo=363529/300/600 #1kgFMあたり地下サイロ価額,地下サイロ300m^3の単価，1m^3=180kgトウモロコシ換算
cost_of_silo_crane=279571 #サイロクレーン

#機械調達部門_変数設定：機械台数
model.number_of_machine_tractor_130ps=Var(initialize=1,within=NonNegativeIntegers) #１3０psトラクター
model.number_of_machine_tractor_160ps=Var(initialize=1,within=NonNegativeIntegers) #１6０psトラクター
model.number_of_machine_chemper_head3=Var(initialize=1,within=NonNegativeIntegers) #3条ケンパーヘッド
model.number_of_machine_wheel_loader_bucket=Var(initialize=1,within=NonNegativeIntegers) #ホイルローダーバケット
model.number_of_machine_wheel_loader_grab=Var(initialize=1,within=NonNegativeIntegers) #ホイルローダーグラブ
model.number_of_machine_dump=Var(initialize=1,within=NonNegativeIntegers) #ダンプ
model.number_of_machine_compost_spreader=Var(initialize=1,within=NonNegativeIntegers) #マニュアスプレッダー
model.number_of_machine_plow=Var(initialize=1,within=NonNegativeIntegers) #プラウ
model.number_of_machine_corn_planter=Var(initialize=1,within=NonNegativeIntegers) #ジェットシーダー
model.number_of_machine_pressure_roller=Var(initialize=1,within=NonNegativeIntegers) #鎮圧ローラー
model.number_of_machine_passenger=Var(initialize=1,within=NonNegativeIntegers) #乗用管理機
model.number_of_machine_rotary=Var(initialize=1,within=NonNegativeIntegers) #ロータリー
model.number_of_machine_rotary_seeder=Var(initialize=1,within=NonNegativeIntegers) #播種機（イタリアンに使用）
model.number_of_machine_more=Var(initialize=1,within=NonNegativeIntegers) #モア
model.number_of_machine_tedder=Var(initialize=1,within=NonNegativeIntegers) #テッダー
model.number_of_machine_rake=Var(initialize=1,within=NonNegativeIntegers) #レーキ
model.number_of_machine_role_baler=Var(initialize=1,within=NonNegativeIntegers) #ロールベーラー
model.number_of_machine_rapper=Var(initialize=1,within=NonNegativeIntegers) #ラッピングマシン



#機械調達部門_制約：旬別の労働及び機械の需給均衡
##4月中+下旬（イタリアン収穫）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_4_2=Constraint(expr=
                                      (model.number_of_hierd_labor_year+number_of_family_core_labor)*20*sunny_rate*model.day_hour_fodder
                                      >=model.number_of_unit_italian_reap*model.hour_per_unit_italian_reap
                                      +model.number_of_unit_italian_reverse*model.hour_per_unit_italian_reverse
                                      +model.number_of_unit_italian_gather*model.hour_per_unit_italian_gather
                                      +model.number_of_unit_italian_roll*model.hour_per_unit_italian_roll
                                      +model.number_of_unit_italian_rapping*model.hour_per_unit_italian_rapping)
##4月中+下旬（イタリアン収穫）_機械台数：組作業ユニット数は所有機械台数以下
model.const_machine_more_4_2=Constraint(expr=model.number_of_machine_more>=model.number_of_unit_italian_reap)
model.const_machine_tedder_4_2=Constraint(expr=model.number_of_machine_tedder>=model.number_of_unit_italian_reverse)
model.const_machine_rake_4_2=Constraint(expr=model.number_of_machine_rake>=model.number_of_unit_italian_gather)
model.const_machine_role_baker_4_2=Constraint(expr=model.number_of_machine_role_baler>=model.number_of_unit_italian_roll)
model.const_machine_rapping_4_2=Constraint(expr=model.number_of_machine_rapper>=model.number_of_unit_italian_rapping)
model.const_machine_tractor_4_2_reap=Constraint(expr= #モアを牽引するトラクターは160psを使用する
                                                model.number_of_machine_tractor_160ps
                                                >=model.number_of_unit_italian_reap)
model.const_machine_tractor_4_2_reverse=Constraint(expr= #130psか160psどちらかがあればいい．
                                                   model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                   >=model.number_of_unit_italian_reverse)
model.const_machine_tractor_4_2_gather=Constraint(expr= #130psか160psどちらかがあればいい．
                                                  model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                  >=model.number_of_unit_italian_gather)
model.const_machine_tractor_4_2_roll=Constraint(expr= #130psか160psどちらかがあればいい．
                                                model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                >=model.number_of_unit_italian_roll)
model.const_machine_tractor_4_2_rapping=Constraint(expr= #130psか160psどちらかがあればいい．
                                                   model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                   >=model.number_of_unit_italian_rapping)
##4月中+下旬（イタリアン収穫）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_more_4_2=Constraint(expr=
                                             model.number_of_machine_more*20*sunny_rate*model.day_hour_fodder
                                             >=model.number_of_unit_italian_reap*model.hour_per_unit_italian_reap)
model.const_machine_hour_tedder_4_2=Constraint(expr=
                                               model.number_of_machine_tedder*20*sunny_rate*model.day_hour_fodder
                                               >=model.number_of_unit_italian_reverse*model.hour_per_unit_italian_reverse)
model.const_machine_hour_rake_4_2=Constraint(expr=
                                             model.number_of_machine_rake*20*sunny_rate*model.day_hour_fodder
                                             >=model.number_of_unit_italian_gather*model.hour_per_unit_italian_gather)
model.const_machine_hour_role_baker_4_2=Constraint(expr=
                                                   model.number_of_machine_role_baler*20*sunny_rate*model.day_hour_fodder
                                                   >=model.number_of_unit_italian_roll*model.hour_per_unit_italian_roll)
model.const_machine_hour_rapping_4_2=Constraint(expr=
                                                model.number_of_machine_rapper*20*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_rapping*model.hour_per_unit_italian_rapping)
model.const_machine_hour_tractor_4_2_2=Constraint(expr=
                                                model.number_of_machine_tractor_160ps*20*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_reap*model.hour_per_unit_italian_reap)
model.const_machine_hour_tractor_4_2_1=Constraint(expr=
                                                (model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps)
                                                  *20*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_reap*model.hour_per_unit_italian_reap
                                                +model.number_of_unit_italian_reverse*model.hour_per_unit_italian_reverse
                                                +model.number_of_unit_italian_gather*model.hour_per_unit_italian_gather
                                                +model.number_of_unit_italian_roll*model.hour_per_unit_italian_roll
                                                +model.number_of_unit_italian_rapping*model.hour_per_unit_italian_rapping)



##５月中旬+下旬（トウモロコシ圃場準備・播種）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_5_2=Constraint(expr=
                                      (model.number_of_hierd_labor_year+number_of_family_core_labor)*20*sunny_rate*model.day_hour_fodder
                                      >=model.number_of_unit_corn_compost_spread*model.hour_per_unit_corn_compost_spread
                                      +model.number_of_unit_corn_plow*model.hour_per_unit_corn_plow
                                      +model.number_of_unit_corn_rotary*model.hour_per_unit_corn_rotary
                                      +model.number_of_unit_corn_plant*model.hour_per_unit_corn_plant
                                      +model.number_of_unit_corn_roller*model.hour_per_unit_corn_roller)
##５月中旬+下旬（トウモロコシ圃場準備・播種）_機械台数：組作業ユニット数は所有機械台数以下
model.const_machine_spreader_5_2=Constraint(expr=
                                            model.number_of_machine_compost_spreader>=model.number_of_unit_corn_compost_spread)
model.const_machine_plow_5_2=Constraint(expr=
                                        model.number_of_machine_plow>=model.number_of_unit_corn_plow)
model.const_machine_rotary_5_2=Constraint(expr=
                                          model.number_of_machine_rotary>=model.number_of_unit_corn_rotary)
model.const_machine_planter_5_2=Constraint(expr=
                                           model.number_of_machine_corn_planter>=model.number_of_unit_corn_plant)
model.const_machine_pressure_roller_5_2=Constraint(expr=
                                                   model.number_of_machine_pressure_roller>=model.number_of_unit_corn_roller)
model.const_machine_tractor_5_2_spreader=Constraint(expr= #130psか160psどちらかがあればいい．
                                                    model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                    >=model.number_of_unit_corn_compost_spread)
model.const_machine_tractor_5_2_plow=Constraint(expr= #130psか160psどちらかがあればいい．
                                                model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                >=model.number_of_unit_corn_plow)
model.const_machine_tractor_5_2_rotary=Constraint(expr= #130psか160psどちらかがあればいい．
                                                  model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                  >=model.number_of_unit_corn_rotary)
model.const_machine_tractor_5_2_plant=Constraint(expr= #130psか160psどちらかがあればいい．
                                                 model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                 >=model.number_of_unit_corn_plant)
model.const_machine_tractor_5_2_roller=Constraint(expr= #130psか160psどちらかがあればいい．
                                                  model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                  >=model.number_of_unit_corn_roller)
##５月中旬+下旬（トウモロコシ圃場準備・播種）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_spreader_5_2=Constraint(expr=
                                                 model.number_of_machine_compost_spreader*20*sunny_rate*model.day_hour_fodder
                                                 >=model.number_of_unit_corn_compost_spread*model.hour_per_unit_corn_compost_spread)
model.const_machine_hour_plow_5_2=Constraint(expr=
                                             model.number_of_machine_plow*20*sunny_rate*model.day_hour_fodder
                                             >=model.number_of_unit_corn_plow*model.hour_per_unit_corn_plow)
model.const_machine_hour_rotary_5_2=Constraint(expr=
                                               model.number_of_machine_rotary*20*sunny_rate*model.day_hour_fodder
                                               >=model.number_of_unit_corn_rotary*model.hour_per_unit_corn_rotary)
model.const_machine_hour_plant_5_2=Constraint(expr=
                                              model.number_of_machine_corn_planter*20*sunny_rate*model.day_hour_fodder
                                              >=model.number_of_unit_corn_plant*model.hour_per_unit_corn_plant)
model.const_machine_hour_roller_5_2=Constraint(expr=
                                               model.number_of_machine_pressure_roller*20*sunny_rate*model.day_hour_fodder
                                               >=model.number_of_unit_corn_roller*model.hour_per_unit_corn_roller)
model.const_machine_hour_tractor_5_2=Constraint(expr=
                                                (model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps)
                                                *20*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_corn_compost_spread*model.hour_per_unit_corn_compost_spread
                                               +model.number_of_unit_corn_plow*model.hour_per_unit_corn_plow
                                               +model.number_of_unit_corn_rotary*model.hour_per_unit_corn_rotary
                                               +model.number_of_unit_corn_plant*model.hour_per_unit_corn_plant
                                               +model.number_of_unit_corn_roller*model.hour_per_unit_corn_roller)



##6月上旬（トウモロコシ除草作業1回目）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_6_1=Constraint(expr=
                                      (model.number_of_hierd_labor_year+number_of_family_core_labor)*10*sunny_rate*model.day_hour_fodder
                                      >=model.number_of_unit_corn_herbicide1*model.hour_per_unit_corn_herbicide1)                                
##6月上旬（トウモロコシ除草作業1回目）_機械台数：組作業ユニット数は所有機械台数以下
model.const_machine_passenger_6_1=Constraint(expr=
                                             model.number_of_machine_passenger
                                             >=model.number_of_unit_corn_herbicide1)
##6月上旬（トウモロコシ除草作業1回目）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_passenger_6_1=Constraint(expr=
                                                  model.number_of_machine_passenger*10*sunny_rate*model.day_hour_fodder
                                                  >=model.number_of_unit_corn_herbicide1*model.hour_per_unit_corn_herbicide1)



##6月下旬（トウモロコシ除草作業２回目）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_6_3=Constraint(expr=
                                      (model.number_of_hierd_labor_year+number_of_family_core_labor)*10*sunny_rate*model.day_hour_fodder
                                      >=model.number_of_unit_corn_herbicide2*model.hour_per_unit_corn_herbicide2)                                
##6月下旬（トウモロコシ除草作業２回目）_機械台数：組作業ユニット数は所有機械台数以下
model.const_machine_passenger_6_3=Constraint(expr=
                                             model.number_of_machine_passenger
                                             >=model.number_of_unit_corn_herbicide2)
##6月下旬（トウモロコシ除草作業２回目）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_passenger_6_3=Constraint(expr=
                                                  model.number_of_machine_passenger*10*sunny_rate*model.day_hour_fodder
                                                  >=model.number_of_unit_corn_herbicide2*model.hour_per_unit_corn_herbicide2)



##7月下旬+8月上旬+9月上旬（飼料用稲収穫）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_7_3=Constraint(expr=
                                      (model.number_of_hierd_labor_year+number_of_family_core_labor)*30*sunny_rate*model.day_hour_fodder
                                      >=model.number_of_unit_rice_silage_reap*model.hour_per_unit_rice_silage_reap
                                      +model.number_of_unit_rice_silage_reverse*model.hour_per_unit_rice_silage_reverse
                                      +model.number_of_unit_rice_silage_gather*model.hour_per_unit_rice_silage_gather
                                      +model.number_of_unit_rice_silage_roll*model.hour_per_unit_rice_silage_roll
                                      +model.number_of_unit_rice_silage_rapping*model.hour_per_unit_rice_silage_rapping)
##7月下旬+8月上旬+9月上旬（飼料用稲収穫）_機械台数：組作業ユニット数は所有機械台数以下
model.const_machine_more_7_3=Constraint(expr=
                                        model.number_of_machine_more
                                        >=model.number_of_unit_rice_silage_reap)
model.const_machine_tedder_7_3=Constraint(expr=
                                          model.number_of_machine_tedder
                                          >=model.number_of_unit_rice_silage_reverse)
model.const_machine_rake_7_3=Constraint(expr=
                                        model.number_of_machine_rake
                                        >=model.number_of_unit_rice_silage_gather)
model.const_machine_role_baker_7_3=Constraint(expr=
                                              model.number_of_machine_role_baler
                                              >=model.number_of_unit_rice_silage_roll)
model.const_machine_rapping_7_3=Constraint(expr=
                                           model.number_of_machine_rapper
                                           >=model.number_of_unit_rice_silage_rapping)
model.const_machine_tractor_7_3_reap=Constraint(expr= #モアを牽引するトラクターは160psを使用する
                                               model.number_of_machine_tractor_160ps
                                                >=model.number_of_unit_rice_silage_reap)
model.const_machine_tractor_7_3_reverse=Constraint(expr= #130psか160psどちらかがあればいい.
                                                   model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                   >=model.number_of_unit_rice_silage_reverse)
model.const_machine_tractor_7_3_gather=Constraint(expr= #130psか160psどちらかがあればいい.
                                                  model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                  >=model.number_of_unit_rice_silage_gather)
model.const_machine_tractor_7_3_roll=Constraint(expr= #130psか160psどちらかがあればいい.
                                                model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                >=model.number_of_unit_rice_silage_roll)
model.const_machine_tractor_7_3_rapping=Constraint(expr= #130psか160psどちらかがあればいい.
                                                   model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                   >=model.number_of_unit_rice_silage_rapping)
##7月下旬+8月上旬+9月上旬（飼料用稲収穫）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_more_7_3=Constraint(expr=
                                             model.number_of_machine_more*30*sunny_rate*model.day_hour_fodder
                                             >=model.number_of_unit_rice_silage_reap*model.hour_per_unit_rice_silage_reap)
model.const_machine_hour_tedder_7_3=Constraint(expr=
                                               model.number_of_machine_tedder*30*sunny_rate*model.day_hour_fodder
                                               >=model.number_of_unit_rice_silage_reverse*model.hour_per_unit_rice_silage_reverse)
model.const_machine_hour_rake_7_3=Constraint(expr=
                                             model.number_of_machine_rake*30*sunny_rate*model.day_hour_fodder
                                             >=model.number_of_unit_rice_silage_gather*model.hour_per_unit_rice_silage_gather)
model.const_machine_hour_role_baker_7_3=Constraint(expr=
                                                   model.number_of_machine_role_baler*30*sunny_rate*model.day_hour_fodder
                                                   >=model.number_of_unit_rice_silage_roll*model.hour_per_unit_rice_silage_roll)
model.const_machine_hour_rapping_7_3=Constraint(expr=
                                                model.number_of_machine_rapper*30*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_rice_silage_rapping*model.hour_per_unit_rice_silage_rapping)
model.const_machine_hour_tractor_7_3_2=Constraint(expr=
                                               model.number_of_machine_tractor_160ps*30*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_rice_silage_reap*model.hour_per_unit_rice_silage_reap)
model.const_machine_hour_tractor_7_3_1=Constraint(expr=
                                                (model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps)*30*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_rice_silage_reap*model.hour_per_unit_rice_silage_reap
                                                +model.number_of_unit_rice_silage_reverse*model.hour_per_unit_rice_silage_reverse
                                                +model.number_of_unit_rice_silage_gather*model.hour_per_unit_rice_silage_gather
                                                +model.number_of_unit_rice_silage_roll*model.hour_per_unit_rice_silage_roll
                                                +model.number_of_unit_rice_silage_rapping*model.hour_per_unit_rice_silage_rapping)
                   
    

##9月下旬（トウモロコシ収穫）_労働人数（組作業は人数も明示的に示す）：組作業ユニットで必要な人数は延べ労働人数以下
model.const_labor_9_3=Constraint(expr= #自家収穫とコントラ両方の場合
                                 model.number_of_hierd_labor_year
                                 +number_of_family_core_labor
                                 +model.number_of_corn_silage_day_labor_self1
                                 +model.number_of_corn_silage_day_labor_contract1
                                 >=model.number_of_unit_corn_harvest1*3 #自家収穫は3人1組
                                 +model.number_of_unit_corn_harvest_contract1*2) #４人１組（1ユニット）のうちコントラ派遣の2名のオペレーター以外にも２人の労働力が必要
model.const_labor_9_3_1=Constraint(expr= #自家収穫の場合
                                   model.number_of_hierd_labor_year
                                   +number_of_family_core_labor
                                   +model.number_of_corn_silage_day_labor_self1
                                   >=model.number_of_unit_corn_harvest1*3) #自家収穫は3人1組
model.const_labor_9_3_2=Constraint(expr= #コントラの場合
                                   model.number_of_hierd_labor_year
                                   +number_of_family_core_labor
                                   +model.number_of_corn_silage_day_labor_contract1
                                   >=model.number_of_unit_corn_harvest_contract1*2) #４人１組（1ユニット）のうちコントラ派遣の2名のオペレーター以外にも２人の労働力が必要
##9月下旬（トウモロコシ収穫）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_9_3=Constraint(expr=
                                      (model.number_of_hierd_labor_year
                                       +number_of_family_core_labor
                                       +model.number_of_corn_silage_day_labor_self1
                                       +model.number_of_corn_silage_day_labor_contract1)*10*sunny_rate*model.day_hour_fodder
                                      >=model.number_of_unit_corn_harvest1*model.hour_per_unit_corn_harvest1*3 #自家収穫は3人1組
                                      +model.number_of_unit_corn_harvest_contract1*model.hour_per_unit_corn_harvest_contract1*2) #４人１組（1ユニット）のうちコントラ派遣の2名のオペレーター以外にも２人の労働力が必要
##9月下旬（トウモロコシ収穫）_機械台数：組作業ユニット数は機械台数以下
model.const_machine_chemperhead3_9_3=Constraint(expr= #自家収穫3条ケンパー
                                               model.number_of_machine_chemper_head3>=model.number_of_unit_corn_harvest1) 
model.const_machine_dump_9_3=Constraint(expr= #自家収穫とコントラ
                                        model.number_of_machine_dump
                                        >=model.number_of_unit_corn_harvest1+model.number_of_unit_corn_harvest_contract1) 
model.const_machine_tractor_9_3=Constraint(expr= #トラクター, 130psか160psどちらかがあればいい．
                                                model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                >=model.number_of_unit_corn_harvest1)
##9月下旬（トウモロコシ収穫）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_chemperhead_9_3=Constraint(expr=
                                                    (model.number_of_machine_chemper_head3)*10*sunny_rate*model.day_hour_fodder
                                                    >=model.number_of_unit_corn_harvest1*model.hour_per_unit_corn_harvest1)
model.const_machine_hour_dump_9_3=Constraint(expr=
                                             model.number_of_machine_dump*10*sunny_rate*model.day_hour_fodder
                                             >=model.number_of_unit_corn_harvest1*model.hour_per_unit_corn_harvest1
                                             +model.number_of_unit_corn_harvest_contract1*model.hour_per_unit_corn_harvest_contract1)
model.const_machine_hour_tractor_9_3=Constraint(expr=
                                                (model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps)
                                                *10*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_corn_harvest1*model.hour_per_unit_corn_harvest1)



##10月上旬（トウモロコシ収穫+イタリアン播種）_労働人数（組作業は人数も明示的に示す）：組作業ユニットで必要な人数は延べ労働人数以下
model.const_labor_10_1=Constraint(expr= #自家収穫とコントラ両方の場合
                                  model.number_of_hierd_labor_year
                                  +number_of_family_core_labor
                                  +model.number_of_corn_silage_day_labor_self2
                                  +model.number_of_corn_silage_day_labor_contract2
                                  >=model.number_of_unit_corn_harvest2*3 #自家収穫は3人1組
                                  +model.number_of_unit_corn_harvest_contract2*2) #４人１組（1ユニット）のうちコントラ派遣の2名のオペレーター以外にも２人の労働力が必要
model.const_labor_10_1_1=Constraint(expr= #自家収穫の場合
                                    model.number_of_hierd_labor_year
                                    +number_of_family_core_labor
                                    +model.number_of_corn_silage_day_labor_self2
                                    >=model.number_of_unit_corn_harvest2*3) #自家収穫は3人1組
model.const_labor_10_1_2=Constraint(expr= #コントラの場合
                                    model.number_of_hierd_labor_year
                                    +number_of_family_core_labor
                                    +model.number_of_corn_silage_day_labor_contract2
                                    >=model.number_of_unit_corn_harvest_contract2*2) #４人１組（1ユニット）のうちコントラ派遣の2名のオペレーター以外にも２人の労働力が必要
##10月上旬（トウモロコシ収穫+イタリアン播種）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_10_1=Constraint(expr=
                                       (model.number_of_hierd_labor_year+number_of_family_core_labor
                                        +model.number_of_corn_silage_day_labor_self2
                                        +model.number_of_corn_silage_day_labor_contract2)*10*sunny_rate*model.day_hour_fodder
                                       >=model.number_of_unit_italian_plant1*model.hour_per_unit_italian_plant1
                                       +model.number_of_unit_italian_roller1*model.hour_per_unit_italian_roller1
                                       +model.number_of_unit_corn_harvest2*model.hour_per_unit_corn_harvest2*3
                                       +model.number_of_unit_corn_harvest_contract2*model.hour_per_unit_corn_harvest_contract2*2)                                         
##10月上旬（トウモロコシ収穫+イタリアン播種）_機械台数：組作業ユニット数は機械台数以下
model.const_machine_rotery_10_1=Constraint(expr=
                                           model.number_of_machine_rotary>=model.number_of_unit_italian_plant1)
model.const_machine_rotery_seeder_10_1=Constraint(expr=
                                                  model.number_of_machine_rotary_seeder>=model.number_of_unit_italian_plant1)
model.const_machine_pressure_roller_10_1=Constraint(expr=
                                                    model.number_of_machine_pressure_roller>=model.number_of_unit_italian_roller1)
model.const_machine_chemperhead3_10_1=Constraint(expr=
                                                model.number_of_machine_chemper_head3>=model.number_of_unit_corn_harvest2)
model.const_machine_dump_10_1=Constraint(expr=
                                         model.number_of_machine_dump
                                         >=model.number_of_unit_corn_harvest2+model.number_of_unit_corn_harvest_contract2)
model.const_machine_tractor_10_1=Constraint(expr= #130psか160psどちらかがあればいい．
                                            model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                            >=model.number_of_unit_corn_harvest2)
model.const_machine_tractor_10_1_plant=Constraint(expr= #130psか160psどちらかがあればいい．
                                                  model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                  >=model.number_of_unit_italian_plant1)
model.const_machine_tractor_10_1_roller=Constraint(expr= #130psか160psどちらかがあればいい．
                                                   model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                   >=model.number_of_unit_italian_roller1)
##10月上旬（トウモロコシ収穫+イタリアン播種）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_chemperhead_10_1=Constraint(expr= # 収穫機（トウモロコシ）
                                                     (model.number_of_machine_chemper_head3) *10*sunny_rate*model.day_hour_fodder
                                                     >=model.number_of_unit_corn_harvest2*model.hour_per_unit_corn_harvest2)
model.const_machine_hour_dump_10_1=Constraint(expr= # ダンプ（トウモロコシ）
                                              model.number_of_machine_dump*10*sunny_rate*model.day_hour_fodder
                                              >=model.number_of_unit_corn_harvest2*model.hour_per_unit_corn_harvest2
                                              +model.number_of_unit_corn_harvest_contract2*model.hour_per_unit_corn_harvest_contract2)
model.const_machine_hour_rotary_10_1=Constraint(expr= # ロータリー（イタリアン）
                                                model.number_of_machine_rotary*10*sunny_rate*model.day_hour_fodder
                                                 >=model.number_of_unit_italian_plant1*model.hour_per_unit_italian_plant1)
model.const_machine_hour_seeder_10_1=Constraint(expr= # 播種機（イタリアン）
                                                model.number_of_machine_rotary_seeder*10*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_plant1*model.hour_per_unit_italian_plant1)
model.const_machine_hour_roller_10_1=Constraint(expr= # 鎮圧ローラー（イタリアン）
                                                model.number_of_machine_pressure_roller*10*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_roller1*model.hour_per_unit_italian_roller1)                                             
model.const_machine_hour_tractor_10_1=Constraint(expr= # トラクター（トウモロコシとイタリアン）
                                                 (model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps)
                                                 *10*sunny_rate*model.day_hour_fodder
                                                 >=model.number_of_unit_italian_plant1*model.hour_per_unit_italian_plant1
                                                 +model.number_of_unit_italian_roller1*model.hour_per_unit_italian_roller1
                                                 +model.number_of_unit_corn_harvest2*model.hour_per_unit_corn_harvest2)   



##10月中旬（イタリアン播種）_労働時間：組作業ユニットの作業時間は延べ労働可能時間以下
model.const_labor_hour_10_2=Constraint(expr=
                                       (model.number_of_hierd_labor_year+number_of_family_core_labor)*10*sunny_rate*model.day_hour_fodder
                                       >=model.number_of_unit_italian_plant2*model.hour_per_unit_italian_plant2
                                       +model.number_of_unit_italian_roller2*model.hour_per_unit_italian_roller2)
##10月中旬（イタリアン播種）_機械台数：組作業ユニット数は機械台数以下
model.const_machine_rotery_10_2=Constraint(expr=
                                           model.number_of_machine_rotary>=model.number_of_unit_italian_plant2)
model.const_machine_rotery_seeder_10_2=Constraint(expr=
                                                  model.number_of_machine_rotary_seeder>=model.number_of_unit_italian_plant2)
model.const_machine_pressure_roller_10_2=Constraint(expr=
                                                    model.number_of_machine_pressure_roller>=model.number_of_unit_italian_roller2)
model.const_machine_tractor_10_2_plant=Constraint(expr= #130psか160psどちらかがあればいい．
                                                  model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                  >=model.number_of_unit_italian_plant2)
model.const_machine_tractor_10_2_roller=Constraint(expr= #130psか160psどちらかがあればいい．
                                                   model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps
                                                   >=model.number_of_unit_italian_roller2)
##10月中旬（イタリアン播種）_機械稼働時間：組作業ユニットの作業時間は延べ機械稼働可能時間以下
model.const_machine_hour_rotary_10_2=Constraint(expr=
                                                model.number_of_machine_rotary*10*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_plant2*model.hour_per_unit_italian_plant2)
model.const_machine_hour_seeder_10_2=Constraint(expr=
                                                model.number_of_machine_rotary_seeder*10*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_plant2*model.hour_per_unit_italian_plant2)
model.const_machine_hour_roller_10_2=Constraint(expr=
                                                model.number_of_machine_pressure_roller*10*sunny_rate*model.day_hour_fodder
                                                >=model.number_of_unit_italian_roller2*model.hour_per_unit_italian_roller2)
model.const_machine_hour_tractor_10_2=Constraint(expr=
                                                 (model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps)
                                                  *10*sunny_rate*model.day_hour_fodder
                                                 >=model.number_of_unit_italian_plant2*model.hour_per_unit_italian_plant2
                                                 +model.number_of_unit_italian_roller2*model.hour_per_unit_italian_roller2)    


##トラクター台数の上限
model.const_machine_hold=Constraint(expr=
                                    model.number_of_machine_tractor_130ps+model.number_of_machine_tractor_160ps<=3)










#飼養管理部門_前提：固定費用（機械施設年償却費）
cost_of_cowshed_main=(3594204.429)/77 #搾乳舎（パイプライン込み）
cost_of_cowshed_sub=(600615)/50 #育成舎
cost_of_fixed_machine=(1513664.835) #機械倉庫, 自動給餌器, TMRミキサー, バルククーラー
cost_of_manure=(852369.2631)/77+500*10000/7/77 #堆肥舎＋専用ローダー
#飼養管理部門_前提：労働時間(1日１頭労働力１人あたり)
hour_per_cattle_milking=0.003937008+0.005249344+0.003937008+0.011111111+0.133333333+0.003571429 #自動給餌機使用
hour_per_cattle_dry=0.007462687+0.007462687 
hour_per_cattle_raised=0.007462687+0.007462687

#飼養管理部門_変数設定：飼養管理に係る1日当たり労働時間
model.day_hour_cattle=Var(initialize=6,within=NonNegativeReals) 

#飼養管理部門_制約：労働時間
model.const_fodder_cattle=Constraint(expr=model.day_hour_fodder+model.day_hour_cattle==12) #1日１２時間の労働時間帯を飼養管理と飼料生産に配分
model.const_cattle_work=Constraint(expr=model.day_hour_cattle<=6) #飼養管理時間は6時間以下
model.const_cattle_feed=Constraint(expr= #飼養管理時間は投下労働の合計時間以下
                                   hour_per_cattle_milking*model.cattle_milking
                                   +hour_per_cattle_dry*model.cattle_dry
                                   +hour_per_cattle_raised*model.cattle_raised_total
                                   <=(number_of_family_core_labor
                                      +number_of_family_sub_labor
                                      +model.number_of_hierd_labor_year)*model.day_hour_cattle) 
#飼養管理部門_制約：作業に必要な機械台数
model.const_machine_hold4=Constraint(expr=model.number_of_machine_wheel_loader_bucket==1)
model.const_machine_hold5=Constraint(expr=model.number_of_machine_wheel_loader_grab==1)   
model.const_machine_hold6=Constraint(expr=model.number_of_machine_dump>=1) 










#目標関数
model.obj=Objective(expr=model.revenue-model.cost, sense= maximize)
#粗収入
model.const_revenue=Constraint(expr=
                               model.revenue
                               ==model.cattle_milking*milk*365*price_milk
                               +model.calf_xy*price_calf_xy
                               +(model.calf_xx-model.cattle_raised)*price_calf_xx)
#うち雑収入
model.const_miscRev=Constraint(expr=
                               model.miscRev
                               ==model.calf_xy*price_calf_xy
                               +(model.calf_xx-model.cattle_raised)*price_calf_xx)
#総費用
model.const_cost=Constraint(expr=
                            model.cost
                            ==model.cost_feed
                            +variable_cost_of_corn*model.area_corn_silage
                            +variable_cost_of_italian*model.area_italian
                            +variable_cost_of_rice_silage*model.area_rice_silage
                            +variable_cost_of_cattle*model.cattle
                            +model.number_of_hierd_labor_year*cost_of_hierd_labor
                            +model.area_contract_corn_harvest*cost_of_contract_corn_harvest
                            +(model.area_contract_corn_herbicide1+model.area_contract_corn_herbicide2)*cost_of_contract_corn_herbicide
                            +model.number_of_unit_corn_harvest1
                                *model.hour_per_unit_corn_harvest1
                                *model.number_of_corn_silage_day_labor_self1
                                *cost_of_day_labor
                            +model.number_of_unit_corn_harvest2
                                *model.hour_per_unit_corn_harvest2
                                *model.number_of_corn_silage_day_labor_self2
                                *cost_of_day_labor
                            +model.number_of_unit_corn_harvest_contract1
                                *model.hour_per_unit_corn_harvest_contract1
                                *model.number_of_corn_silage_day_labor_contract1
                                *cost_of_day_labor  
                            +model.number_of_unit_corn_harvest_contract2
                                *model.hour_per_unit_corn_harvest_contract2
                                *model.number_of_corn_silage_day_labor_contract2
                                *cost_of_day_labor
                            +model.cost_fixed
                            +model.cattle_buy*900000/4)
#うち購入飼料費
model.const_cost_feed=Constraint(expr=
                                 model.cost_feed
                                 ==model.timothy_menue*timothy_cost*model.cattle_milking*365
                                 +model.alfalfa_menue*alfalfa_cost*model.cattle_milking*365
                                 +model.mixed_menue*mixed_cost*model.cattle_milking*365
                                 +model.soybean_menue*soybean_meal_cost*model.cattle_milking*365
                                 +model.flakedcorn_menue*flakedcorn_cost*model.cattle_milking*365
                                 +model.timothy_menue_raised*timothy_cost*model.cattle_raised_total*365
                                 +model.alfalfa_menue_raised*alfalfa_cost*model.cattle_raised_total*365
                                 +model.soybean_menue_raised*soybean_meal_cost*model.cattle_raised_total*365
                                 +model.flakedcorn_menue_raised*flakedcorn_cost*model.cattle_raised_total*365
                                 +model.timothy_menue_dry*timothy_cost*model.cattle_dry*365
                                 +model.alfalfa_menue_dry*alfalfa_cost*model.cattle_dry*365
                                 +model.mixed_menue_dry*mixed_cost*model.cattle_dry*365
                                 +model.soybean_menue_dry*soybean_meal_cost*model.cattle_dry*365
                                 +model.flakedcorn_menue_dry*flakedcorn_cost*model.cattle_dry*365)
#うち固定費用
model.const_fixed_cost=Constraint(expr=
                                 model.cost_fixed==
                                 model.number_of_machine_tractor_130ps*cost_of_machine_tractor_130ps
                                +model.number_of_machine_tractor_160ps*cost_of_machine_tractor_160ps
                                +model.number_of_machine_chemper_head3*cost_of_machine_chemper_head3
                                +model.number_of_machine_wheel_loader_bucket*cost_of_machine_wheel_loader_bucket
                                +model.number_of_machine_wheel_loader_grab*cost_of_machine_wheel_loader_grab
                                +model.number_of_machine_dump*cost_of_machine_dump
                                +model.number_of_machine_compost_spreader*cost_of_machine_dump
                                +model.number_of_machine_plow*cost_of_machine_plow
                                +model.number_of_machine_corn_planter*cost_of_machine_corn_planter
                                +model.number_of_machine_pressure_roller*cost_of_machine_pressure_roller
                                +model.number_of_machine_passenger*cost_of_machine_passenger
                                +model.number_of_machine_rotary*cost_of_machine_rotary
                                +model.number_of_machine_rotary_seeder*cost_of_machine_rotary_seeder
                                +model.number_of_machine_more*cost_of_machine_more
                                +model.number_of_machine_tedder*cost_of_machine_tedder
                                +model.number_of_machine_rake*cost_of_machine_rake
                                +model.number_of_machine_role_baler*cost_of_machine_role_baler
                                +model.number_of_machine_rapper*cost_of_machine_rapper
                                +model.cattle*cost_of_manure
                                +model.cattle*cost_of_cowshed_main
                                +model.cattle_raised_total*cost_of_cowshed_sub
                                +cost_of_fixed_machine
                                +cost_of_silo*(model.area_corn_silage-model.area_contract_corn_harvest)*yield_corn
                                +cost_of_silo_crane*model.number_of_machine_chemper_head3)
#うち育成牛購入飼料費
model.const_cost_feed_raised=Constraint(expr=
                                 model.cost_raised_feed
                                 ==model.timothy_menue_raised*timothy_cost*model.cattle_raised_total*365
                                 +model.alfalfa_menue_raised*alfalfa_cost*model.cattle_raised_total*365
                                 +model.soybean_menue_raised*soybean_meal_cost*model.cattle_raised_total*365
                                 +model.flakedcorn_menue_raised*flakedcorn_cost*model.cattle_raised_total*365)
#うち育成牛固定費用
model.const_cost_fixed_raised=Constraint(expr=model.cost_raised_fixed==model.cattle_raised_total*cost_of_cowshed_sub)
#うち導入牛購入費用
model.const_cost_cattle_buy=Constraint(expr=model.cost_cattle_buy==model.cattle_buy*900000/4)



# シミュレーション条件
model.simu_riceWCS=Constraint(expr=model.area_rice_silage==2000) #a, 飼料稲の面積, 指定しないときはコメントアウト





opt = SolverFactory("scipampl")
result = opt.solve(model, tee=True)
model.display()

20.379529790201094 14.329082279574495 2853.1739636759094
SCIP version 5.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: SoPlex 3.1.1] [GitHash: b1c222a]
Copyright (C) 2002-2018 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

External codes: 
  Readline EditLine w  GNU library for command line editing (gnu.org/s/readline)
  SoPlex 3.1.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: ab921a5]
  CppAD 20160000.1     Algorithmic Differentiation of C++ algorithms developed by B. Bell (www.coin-or.org/CppAD)
  ZLIB 1.2.11          General purpose compression library by J. Gailly and M. Adler (zlib.net)
  GMP 6.1.2            GNU Multiple Precision Arithmetic Library developed by T. Granlund (gmplib.org)
  ASL                  AMPL Solver Library developed by D. Gay (www.netlib.com/ampl)


number of parameters = 2303
non-default parameter settings:


read problem </var/folders/f8/wxs_vbrd67jcbr9tdd4hlrh80000gn/T/

  0.2s|     1 |     2 |  1512 |     - |2959k|   0 |  13 | 121 | 167 | 119 | 157 |  61 |   0 |  14 | 4.014690e+07 | 3.787102e+06 | 960.10%
Starting spatial branch-and-bound on unbounded variable <t__svar[25]> ([201,1e+20]) - cannot guarantee finite termination.
  0.3s|   100 |    49 |  2865 |  26.1 |3725k|  15 |   - | 121 | 167 |   0 |   0 | 415 |   0 |  63 | 3.924817e+07 | 3.787102e+06 | 936.36%
 time | node  | left  |LP iter|LP it/n| mem |mdpt |frac |vars |cons |cols |rows |cuts |confs|strbr|  dualbound   | primalbound  |  gap   
  0.4s|   200 |    81 |  3396 |  15.6 |3913k|  21 |   - | 121 | 167 |   0 |   0 | 562 |   0 |  63 | 3.770167e+07 | 3.787102e+06 | 895.53%
  0.5s|   300 |   109 |  3769 |  11.6 |4053k|  25 |   0 | 121 | 167 | 119 | 174 | 681 |   0 |  65 | 3.770167e+07 | 3.787102e+06 | 895.53%
  0.6s|   400 |   133 |  4056 |   9.4 |4518k|  25 |   0 | 121 | 167 | 119 | 192 | 794 |   0 |  69 | 3.770167e+07 | 3.787102e+06 | 895.53%
  0.8s|   500 |   177 |  4784 |   9.0 |4711k|  29

C 7.7s|  5199 |   234 |124346 |  23.9 |  22M|  51 |   - | 121 | 167 | 119 | 246 |  15k|   0 | 430 | 2.538765e+07 | 2.307953e+07 |  10.00%
  7.7s|  5200 |   235 |124352 |  23.9 |  22M|  51 |   0 | 121 | 167 | 119 | 235 |  15k|   0 | 430 | 2.538765e+07 | 2.307953e+07 |  10.00%
r 7.7s|  5277 |   258 |124613 |  23.6 |  22M|  53 |   - | 121 | 167 | 119 | 263 |  15k|   0 | 430 | 2.536705e+07 | 2.307953e+07 |   9.91%
  7.7s|  5300 |   245 |124673 |  23.5 |  22M|  53 |   1 | 121 | 167 | 119 | 228 |  15k|   0 | 430 | 2.536705e+07 | 2.307953e+07 |   9.91%
 time | node  | left  |LP iter|LP it/n| mem |mdpt |frac |vars |cons |cols |rows |cuts |confs|strbr|  dualbound   | primalbound  |  gap   
  7.8s|  5400 |   267 |125027 |  23.1 |  22M|  53 |   0 | 121 | 167 | 119 | 254 |  16k|   0 | 430 | 2.525257e+07 | 2.307953e+07 |   9.42%
r 7.8s|  5427 |   274 |125081 |  23.0 |  22M|  53 |   - | 121 | 167 | 119 | 259 |  16k|   0 | 430 | 2.525257e+07 | 2.307953e+07 |   9.42%
r 7.8s|  5430 |   271 |125084 |  2

 10.8s| 10500 |  1215 |138656 |  13.2 |  24M|  97 |   - | 121 | 167 | 119 | 279 |  22k|   0 | 562 | 2.347397e+07 | 2.307953e+07 |   1.71%
 10.8s| 10600 |  1207 |138864 |  13.1 |  24M|  97 |   0 | 121 | 167 | 119 | 271 |  22k|   0 | 564 | 2.346151e+07 | 2.307953e+07 |   1.66%
 10.9s| 10700 |  1211 |139092 |  13.0 |  24M|  97 |   0 | 121 | 167 | 119 | 275 |  22k|   0 | 565 | 2.345169e+07 | 2.307953e+07 |   1.61%
 10.9s| 10800 |  1213 |139368 |  12.9 |  24M|  97 |   - | 121 | 167 | 119 | 286 |  22k|   0 | 567 | 2.339872e+07 | 2.307953e+07 |   1.38%
 11.0s| 10900 |  1207 |139619 |  12.8 |  24M|  97 |   0 | 121 | 167 | 119 | 290 |  22k|   0 | 572 | 2.338410e+07 | 2.307953e+07 |   1.32%
 11.1s| 11000 |  1237 |139853 |  12.7 |  24M|  97 |   - | 121 | 167 | 119 | 285 |  23k|   0 | 572 | 2.338100e+07 | 2.307953e+07 |   1.31%
 11.1s| 11100 |  1239 |140066 |  12.6 |  24M|  97 |   0 | 121 | 167 | 119 | 287 |  23k|   0 | 575 | 2.337914e+07 | 2.307953e+07 |   1.30%
 time | node  | left  |LP iter|LP 

 13.9s| 16300 |  1249 |151197 |   9.3 |  26M| 101 |   0 | 121 | 167 | 119 | 298 |  30k|   0 | 586 | 2.308820e+07 | 2.307953e+07 |   0.04%
 13.9s| 16400 |  1247 |151341 |   9.2 |  26M| 101 |   0 | 121 | 167 | 119 | 302 |  30k|   0 | 586 | 2.308820e+07 | 2.307953e+07 |   0.04%
 13.9s| 16500 |  1249 |151505 |   9.2 |  26M| 101 |   - | 121 | 167 | 119 | 306 |  30k|   0 | 586 | 2.308820e+07 | 2.307953e+07 |   0.04%
 14.0s| 16600 |  1255 |151678 |   9.1 |  26M| 101 |   - | 121 | 167 | 119 | 311 |  30k|   0 | 586 | 2.308814e+07 | 2.307953e+07 |   0.04%
 14.1s| 16700 |  1257 |151875 |   9.1 |  26M| 101 |   0 | 121 | 167 | 119 | 267 |  30k|   0 | 586 | 2.308812e+07 | 2.307953e+07 |   0.04%
 14.1s| 16800 |  1225 |152086 |   9.0 |  26M| 101 |   - | 121 | 167 | 119 | 281 |  30k|   0 | 586 | 2.308802e+07 | 2.307953e+07 |   0.04%
 14.2s| 16900 |  1213 |152310 |   9.0 |  26M| 101 |   - | 121 | 167 | 119 | 297 |  30k|   0 | 586 | 2.308755e+07 | 2.307953e+07 |   0.03%
 14.2s| 17000 |  1207 |152513 |   

...

 23.0s| 34100 |  1463 |186942 |   5.5 |  33M| 111 |   0 | 121 | 167 | 119 | 288 |  53k|   0 | 586 | 2.308418e+07 | 2.307953e+07 |   0.02%
 23.0s| 34200 |  1453 |187086 |   5.5 |  33M| 111 |   - | 121 | 167 | 119 | 276 |  53k|   0 | 586 | 2.308418e+07 | 2.307953e+07 |   0.02%
 23.1s| 34300 |  1465 |187223 |   5.5 |  33M| 111 |   0 | 121 | 167 | 119 | 275 |  53k|   0 | 586 | 2.308418e+07 | 2.307953e+07 |   0.02%
 23.1s| 34400 |  1489 |187385 |   5.4 |  33M| 111 |   0 | 121 | 167 | 119 | 287 |  54k|   0 | 586 | 2.308418e+07 | 2.307953e+07 |   0.02%
 23.2s| 34500 |  1467 |187545 |   5.4 |  33M| 111 |   0 | 121 | 167 | 119 | 282 |  54k|   0 | 586 | 2.308417e+07 | 2.307953e+07 |   0.02%
 23.2s| 34600 |  1453 |187714 |   5.4 |  33M| 111 |   0 | 121 | 167 | 119 | 283 |  54k|   0 | 586 | 2.308417e+07 | 2.307953e+07 |   0.02%
 23.3s| 34700 |  1469 |187895 |   5.4 |  33M| 111 |   0 | 121 | 167 | 119 | 274 |  54k|   0 | 586 | 2.308417e+07 | 2.307953e+07 |   0.02%
 23.3s| 34800 |  1503 |188054 |   

 26.1s| 40100 |  1387 |198806 |   5.0 |  35M| 111 |   - | 121 | 167 | 119 | 296 |  61k|   0 | 586 | 2.308374e+07 | 2.307953e+07 |   0.02%
 26.2s| 40200 |  1393 |198965 |   4.9 |  35M| 111 |   - | 121 | 167 | 119 | 296 |  61k|   0 | 586 | 2.308374e+07 | 2.307953e+07 |   0.02%
 26.2s| 40300 |  1397 |199154 |   4.9 |  35M| 111 |   0 | 121 | 167 | 119 | 290 |  61k|   0 | 586 | 2.308374e+07 | 2.307953e+07 |   0.02%
 26.3s| 40400 |  1413 |199374 |   4.9 |  36M| 111 |   0 | 121 | 167 | 119 | 275 |  62k|   0 | 586 | 2.308374e+07 | 2.307953e+07 |   0.02%
 26.3s| 40500 |  1433 |199578 |   4.9 |  36M| 111 |   0 | 121 | 167 | 119 | 284 |  62k|   0 | 586 | 2.308373e+07 | 2.307953e+07 |   0.02%
 26.4s| 40600 |  1453 |199776 |   4.9 |  36M| 111 |   - | 121 | 167 | 119 | 296 |  62k|   0 | 586 | 2.308373e+07 | 2.307953e+07 |   0.02%
 26.4s| 40700 |  1439 |199987 |   4.9 |  36M| 111 |   - | 121 | 167 | 119 | 302 |  62k|   0 | 586 | 2.308373e+07 | 2.307953e+07 |   0.02%
 26.5s| 40800 |  1441 |200240 |   

 29.2s| 46000 |  1523 |211735 |   4.6 |  39M| 111 |   0 | 121 | 167 | 119 | 253 |  69k|   0 | 586 | 2.308362e+07 | 2.307953e+07 |   0.02%
 29.3s| 46100 |  1521 |211947 |   4.6 |  39M| 111 |   - | 121 | 167 | 119 | 301 |  70k|   0 | 586 | 2.308362e+07 | 2.307953e+07 |   0.02%
 29.3s| 46200 |  1505 |212169 |   4.6 |  39M| 111 |   - | 121 | 167 | 119 | 295 |  70k|   0 | 586 | 2.308361e+07 | 2.307953e+07 |   0.02%
 29.4s| 46300 |  1485 |212366 |   4.6 |  39M| 111 |   0 | 121 | 167 | 119 | 297 |  70k|   0 | 586 | 2.308361e+07 | 2.307953e+07 |   0.02%
 29.4s| 46400 |  1475 |212554 |   4.6 |  39M| 111 |   0 | 121 | 167 | 119 | 302 |  70k|   0 | 586 | 2.308361e+07 | 2.307953e+07 |   0.02%
 29.4s| 46500 |  1499 |212753 |   4.6 |  39M| 111 |   - | 121 | 167 | 119 | 287 |  70k|   0 | 586 | 2.308361e+07 | 2.307953e+07 |   0.02%
 29.5s| 46600 |  1527 |212944 |   4.6 |  39M| 111 |   - | 121 | 167 | 119 | 297 |  70k|   0 | 586 | 2.308360e+07 | 2.307953e+07 |   0.02%
 29.5s| 46700 |  1507 |213112 |   

 32.1s| 52000 |   801 |224734 |   4.3 |  40M| 111 |   - | 121 | 168 | 119 | 298 |  78k|   0 | 586 | 2.308242e+07 | 2.307953e+07 |   0.01%
 32.2s| 52100 |   795 |224964 |   4.3 |  40M| 111 |   - | 121 | 168 | 119 | 306 |  78k|   0 | 586 | 2.308242e+07 | 2.307953e+07 |   0.01%
 32.2s| 52200 |   761 |225124 |   4.3 |  40M| 111 |   - | 121 | 168 | 119 | 300 |  78k|   0 | 586 | 2.308242e+07 | 2.307953e+07 |   0.01%
 32.3s| 52300 |   737 |225308 |   4.3 |  40M| 111 |   0 | 121 | 168 | 119 | 296 |  78k|   0 | 586 | 2.308234e+07 | 2.307953e+07 |   0.01%
 32.3s| 52400 |   713 |225584 |   4.3 |  40M| 111 |   - | 121 | 168 | 119 | 283 |  79k|   0 | 586 | 2.308233e+07 | 2.307953e+07 |   0.01%
 32.4s| 52500 |   701 |225825 |   4.3 |  40M| 111 |   0 | 121 | 168 | 119 | 314 |  79k|   0 | 586 | 2.308233e+07 | 2.307953e+07 |   0.01%
 32.4s| 52600 |   691 |226070 |   4.3 |  40M| 111 |   0 | 121 | 168 | 119 | 325 |  79k|   0 | 586 | 2.308233e+07 | 2.307953e+07 |   0.01%
 32.5s| 52700 |   677 |226330 |   

 35.1s| 58000 |   693 |237147 |   4.1 |  42M| 111 |   0 | 121 | 168 | 119 | 288 |  85k|   0 | 586 | 2.308145e+07 | 2.307953e+07 |   0.01%
 35.1s| 58100 |   669 |237379 |   4.1 |  42M| 111 |   - | 121 | 168 | 119 | 283 |  86k|   0 | 586 | 2.308142e+07 | 2.307953e+07 |   0.01%
 35.2s| 58200 |   705 |237483 |   4.1 |  42M| 111 |   0 | 121 | 168 | 119 | 283 |  86k|   0 | 586 | 2.308141e+07 | 2.307953e+07 |   0.01%
 35.2s| 58300 |   691 |237671 |   4.1 |  42M| 111 |   0 | 121 | 168 | 119 | 288 |  86k|   0 | 586 | 2.308137e+07 | 2.307953e+07 |   0.01%
 35.3s| 58400 |   725 |237784 |   4.1 |  42M| 111 |   - | 121 | 167 | 119 | 264 |  86k|   0 | 586 | 2.308137e+07 | 2.307953e+07 |   0.01%
 35.3s| 58500 |   761 |237901 |   4.1 |  42M| 111 |   - | 121 | 167 | 119 | 259 |  86k|   0 | 586 | 2.308137e+07 | 2.307953e+07 |   0.01%
 35.4s| 58600 |   751 |238120 |   4.1 |  42M| 111 |   0 | 121 | 167 | 119 | 296 |  86k|   0 | 586 | 2.308135e+07 | 2.307953e+07 |   0.01%
 35.4s| 58700 |   799 |238325 |   

In [2]:
#計算結果の出力
model.solutions.load_from(result)

for v in model.component_objects(Var, active=True):
   # print("Variable",v)
    varobject=getattr(model,str(v))
    for index in varobject:
        print(v,"   ",varobject[index].value)

revenue     85399259.50076
miscRev     11809701.67196488
cost_feed     14510026.601996578
cost     59730918.42471762
cost_fixed     19150212.659078885
cost_raised_feed     225230.95423930875
cost_raised_fixed     417780.35028704157
cost_cattle_buy     1676862.9799916577
number_of_hierd_labor_year     0.0
cattle     82.80804839464976
cattle_milking     70.06834864162671
cattle_dry     12.73969975302304
xx_rate     0.22749999999999995
calf_xy     59.048508359824396
calf_xx     17.389690162876448
cattle_raised     17.38969016287645
cattle_raised_total     34.7793803257529
cattle_buy     7.452724355518479
number_of_unit_italian_reap     1.0
number_of_unit_italian_reverse     1.0
number_of_unit_italian_gather     1.0
number_of_unit_italian_roll     1.0
number_of_unit_italian_rapping     1.0
number_of_unit_italian_plant1     1.0
number_of_unit_italian_plant2     1.0
number_of_unit_italian_roller1     1.0
number_of_unit_italian_roller2     1.0
number_of_unit_rice_silage_reap     1.0
number_of